# Imports

In [1]:
%matplotlib qt

from IPython import get_ipython
ipython = get_ipython()
ipython.magic("gui qt5") 

import napari
from PIL import Image

import numpy as np
import cv2

import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import matplotlib.colors as colors
import matplotlib.path as mpltpath

from bisect import bisect_left


import time

import pickle as pkl
import h5py
from os.path import sep

import Cell
import pixels_in_roi
import diff_gauss
import scatter_channels
import hist_channels
import m_dist

from skimage.measure import profile_line

C:\Users\JRCLUST\Anaconda3\lib\site-packages\napari\__init__.py:27: UserWarning: 
    napari was tested with QT library `>=5.12.3`.
    The version installed is 5.9.6. Please report any issues with this
    specific QT version at https://github.com/Napari/napari/issues.
    
  warn(message=warn_message)


# Load images

## Raw images

### Specify planes

In [26]:
t0 = time.time()
folder = 'G:{0}Shared drives{0}as_share{0}HCR{0}HCR 3.12{0}S3{0}S3_raw_tiff_images'.format(sep)
plane_nos = range(1, 14)

# base_filename2 = 'S1_dapi_546_488_647_s2z'
base_filename = 'S3_z'
n = len(plane_nos)
print('Number of planes: {0}'.format(n))

# Create 4D array to store images
img = Image.open('{0}{1}{2}{3}_ChS1-T3_ORG.tif'.format(folder, sep, base_filename, str(plane_nos[0]).zfill(2)))

h = img.height
w = img.width

Number of planes: 13


### Load images

In [4]:
t0 = time.time()
folder = 'G:{0}Shared drives{0}as_share{0}HCR{0}HCR 3.12{0}S3{0}S3_raw_tiff_images'.format(sep)
plane_nos = range(1, 10)

# base_filename2 = 'S1_dapi_546_488_647_s2z'
base_filename = 'S3_DAPI_546_647_514_594_2020_03_16__00_43_13_z'
n = len(plane_nos)
print('Number of planes: {0}'.format(n))

# Create 4D array to store images
img = Image.open('{0}{1}{2}{3}_ChS1-T3_ORG.tif'.format(folder, sep, base_filename, str(plane_nos[0]).zfill(2)))

h = img.height
w = img.width
im_array_vip = np.zeros([n, h, w])
im_array_car4 = np.zeros([n, h, w])
im_array_tac2 = np.zeros([n, h, w])
im_array_npy2r = np.zeros([n, h, w])
#im_array_dapi = np.zeros([n, h, w])

print('Size of image in pixels: {0} X {1} X {2}'.format(n, h, w))

for i in range(n):
    print('Loading image {0}, {1} seconds'.format(i + 1, np.round(time.time() - t0)))
    p = plane_nos[i]
    img_vip = Image.open('{0}{1}{2}{3}_ChS1-T3_ORG.tif'.format(folder, sep, base_filename, str(p).zfill(2)))
    img_npy2r = Image.open('{0}{1}{2}{3}_ChS2-T3_ORG.tif'.format(folder, sep, base_filename, str(p).zfill(2)))
    img_car4 = Image.open('{0}{1}{2}{3}_ChS2-T2_ORG.tif'.format(folder, sep, base_filename, str(p).zfill(2)))
    img_tac2 = Image.open('{0}{1}{2}{3}_ChS2-T1_ORG.tif'.format(folder, sep, base_filename, str(p).zfill(2)))
    #img_dapi = Image.open('{0}{1}{2}{3}_ChS1-T1_ORG.tif'.format(folder, sep, base_filename, str(p).zfill(2)))

    try:
        im_array_vip[i, :, :] = np.array(img_vip)
        im_array_car4[i, :, :] = np.array(img_car4)
        im_array_tac2[i, :, :] = np.array(img_tac2)
        im_array_npy2r[i, :, :] = np.array(img_npy2r)
        #im_array_dapi[i, :, :] = np.array(img_dapi)
    except:
        print('Plane {0} could not be loaded'.format(p))
        print('Size of plane {0} in pixels: {0} X {1}'.format(img.height, img.width))
        #im_array_vip = np.delete(im_array_vip, i, axis = 0)
        im_array_car4 = np.delete(im_array_car4, i, axis = 0)
        im_array_tac2 = np.delete(im_array_tac2, i, axis = 0)
        im_array_npy2r = np.delete(im_array_npy2r, i, axis = 0)
        plane_nos.remove(p)
        i -= 1
        n = len(plane_nos)
        continue

del img_vip
del img_car4
del img_tac2
del img_npy2r
#del img_dapi

Number of planes: 9
Size of image in pixels: 9 X 12083 X 11161
Loading image 1, 0.0 seconds
Loading image 2, 24.0 seconds
Loading image 3, 53.0 seconds
Loading image 4, 81.0 seconds
Loading image 5, 104.0 seconds
Loading image 6, 127.0 seconds
Loading image 7, 150.0 seconds
Loading image 8, 171.0 seconds
Loading image 9, 190.0 seconds


## Gaussian blurred images

In [37]:
# %% Load images
t0 = time.time()
folder = 'G:{0}Shared drives{0}as_share{0}HCR{0}HCR 3.12{0}S3{0}S3_blurred_tiff_images'.format(sep)
plane_nos = range(1, 14)

# base_filename2 = 'S1_dapi_546_488_647_s2z'
base_filename = 'S3_z'
n = len(plane_nos)
print('Number of planes: {0}'.format(n))

# Create 4D array to store images
img = Image.open('{0}{1}{2}{3}_vip_5px_gauss_blur.tif'.format(folder, sep, base_filename, str(plane_nos[0]).zfill(2)))

h = img.height
w = img.width
im_array_vip_blur = np.zeros([n, h, w])
# im_array_car4 = np.zeros([n, h, w])
# im_array_tac2 = np.zeros([n, h, w])
# im_array_npy2r = np.zeros([n, h, w])
# im_array_dapi = np.zeros([n, h, w])

print('Size of image in pixels: {0} X {1} X {2}'.format(n, h, w))

for i in range(n):
    print('Loading image {0}, {1} seconds'.format(i + 1, np.round(time.time() - t0)))
    p = plane_nos[i]
    img_vip = Image.open('{0}{1}{2}{3}_vip_5px_gauss_blur.tif'.format(folder, sep, base_filename, str(p).zfill(2)))
#     img_car4 = Image.open('{0}{1}{2}{3}_ChS2-T3_ORG.tif'.format(folder, sep, base_filename, str(p).zfill(2)))
#     img_tac2 = Image.open('{0}{1}{2}{3}_ChS2-T2_ORG.tif'.format(folder, sep, base_filename, str(p).zfill(2)))
#     img_npy2r = Image.open('{0}{1}{2}{3}_ChS2-T1_ORG.tif'.format(folder, sep, base_filename, str(p).zfill(2)))
#     img_dapi = Image.open('{0}{1}{2}{3}_ChS1-T1_ORG.tif'.format(folder, sep, base_filename, str(p).zfill(2)))

    try:
        im_array_vip_blur[i, :, :] = np.array(img_vip)
#         im_array_car4[i, :, :] = np.array(img_car4)
#         im_array_tac2[i, :, :] = np.array(img_tac2)
#         im_array_npy2r[i, :, :] = np.array(img_npy2r)
#         im_array_dapi[i, :, :] = np.array(img_dapi)
    except:
        print('Plane {0} could not be loaded'.format(p))
        print('Size of plane {0} in pixels: {0} X {1}'.format(img.height, img.width))
        im_array_vip = np.delete(im_array_vip, i, axis = 0)
        im_array_car4 = np.delete(im_array_car4, i, axis = 0)
        im_array_tac2 = np.delete(im_array_tac2, i, axis = 0)
        im_array_npy2r = np.delete(im_array_npy2r, i, axis = 0)
        plane_nos.remove(p)
        i -= 1
        n = len(plane_nos)
        continue

del img_vip
# del img_car4
# del img_tac2
# del img_npy2r
# del img_dapi

Number of planes: 13
Size of image in pixels: 13 X 4709 X 12082
Loading image 1, 0.0 seconds
Loading image 2, 1.0 seconds
Loading image 3, 2.0 seconds
Loading image 4, 4.0 seconds
Loading image 5, 5.0 seconds
Loading image 6, 6.0 seconds
Loading image 7, 7.0 seconds
Loading image 8, 8.0 seconds
Loading image 9, 10.0 seconds
Loading image 10, 11.0 seconds
Loading image 11, 12.0 seconds
Loading image 12, 13.0 seconds
Loading image 13, 14.0 seconds


## Background subtracted (rolling ball)

In [ ]:
# Background subtracted (using rolling ball background subtraction in Fiji)
print('npy2r')
im_array_npy2r = np.zeros([n, h, w])
img_file = 'S2_npy2r_bg_sub2'
img = Image.open('{0}\{1}.tif'.format(folder, img_file))
for i in range(n):
    print('Loading image {0}'.format(i + 1))
    p = plane_nos[i]
    try:
        img.seek(p - 1)
        im_array_npy2r[i, :, :] = np.array(img)
        
    except:
        print('Plane {0} could not be loaded'.format(p))
        print('Size of plane {0} in pixels: {0} X {1}'.format(img.height, img.width))
        im_array_npy2r = np.delete(im_array_npy2r, i, axis = 0)
        plane_nos.remove(p)
        i -= 1
        n = len(plane_nos)
        continue

print('car4')
im_array_car4 = np.zeros([n, h, w])
img_file = 'S2_car4_bg_sub2'
img = Image.open('{0}\{1}.tif'.format(folder, img_file))
for i in range(n):
    print('Loading image {0}'.format(i + 1))
    p = plane_nos[i]
    try:
        img.seek(p - 1)
        im_array_car4[i, :, :] = np.array(img)
        
    except:
        print('Plane {0} could not be loaded'.format(p))
        print('Size of plane {0} in pixels: {0} X {1}'.format(img.height, img.width))
        im_array_car4 = np.delete(im_array_car4, i, axis = 0)
        plane_nos.remove(p)
        i -= 1
        n = len(plane_nos)
        continue
        
        
print('tac2')
im_array_tac2 = np.zeros([n, h, w])
img_file = 'S2_tac2_bg_sub2'
img = Image.open('{0}\{1}.tif'.format(folder, img_file))
for i in range(n):
    print('Loading image {0}'.format(i + 1))
    p = plane_nos[i]
    try:
        img.seek(p - 1)
        im_array_tac2[i, :, :] = np.array(img)
        
    except:
        print('Plane {0} could not be loaded'.format(p))
        print('Size of plane {0} in pixels: {0} X {1}'.format(img.height, img.width))
        im_array_tac2 = np.delete(im_array_tac2, i, axis = 0)
        plane_nos.remove(p)
        i -= 1
        n = len(plane_nos)
        continue    

# Render images in napari GUI

## Raw images

In [32]:
viewer.add_image(im_array_tac2, name = 'tac2', colormap = 'green', blending = 'additive')


<Image layer 'Sst' at 0x2421cf7d5c8>

In [12]:
# Full images
t0 = time.time()

viewer = napari.Viewer()
#viewer.add_image(im_array_vip, name = 'vip', colormap = 'cyan', blending = 'additive')
viewer.add_image(im_array_npy2r, name = 'npy2r', colormap = 'red', blending = 'additive')
viewer.add_image(im_array_car4, name = 'car4', colormap = 'green', blending = 'additive')
viewer.add_image(im_array_tac2, name = 'tac2', colormap = 'blue', blending = 'additive')

t1 = time.time() - t0
print('{0} seconds'.format(np.round(t1)))

C:\Users\JRCLUST\Anaconda3\lib\site-packages\scipy\ndimage\interpolation.py:611: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)
C:\Users\JRCLUST\Anaconda3\lib\site-packages\scipy\ndimage\interpolation.py:611: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)
C:\Users\JRCLUST\Anaconda3\lib\site-packages\scipy\ndimage\interpolation.py:611: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


16.0 seconds


## Blurred images

In [9]:
viewer.add_image(im_array_vip_blur, name = 'vip_blur', colormap = 'cyan', blending = 'additive')

<Image layer 'Gad1_blur' at 0x15913608>

## Cell masks

In [103]:
# Cells with high mask variance
var_thresh = 600
indices = np.where(np.logical_or(var_masks[:, 0] > var_thresh, var_masks[:, 1] > var_thresh))[0] + 1
print(indices)

[18 20 24 28 30 35 37 45 48]


In [14]:
mask_layer2 = viewer.add_shapes(name = 'Cell masks individual')
mask_cell_nos = []
cell_mask_nos = {}
t0 = time.time()

indices = list(Cell.cell_data.keys())
# indices = cells + 1
for cell in indices:
    if np.mod(cell, 10) == 0:
        print('Cell {0}, {1} seconds'.format(cell, np.round(time.time() - t0)))
    planes = Cell.cell_data[cell]['z_planes']
    cell_mask_nos[cell] = []
    for plane in planes:
        mask = Cell.cell_data[cell]['masks'][plane]
        mask = np.concatenate((np.ones([mask.shape[0], 1])*plane, mask), axis = 1)
        mask_layer2.add(mask, shape_type = 'polygon', opacity = 0.4, 
                        face_color = 'white', edge_color = 'red', edge_width = 5)
        cell_mask_nos[cell] = np.append(cell_mask_nos[cell], len(mask_cell_nos) - 1)
        mask_cell_nos = np.append(mask_cell_nos, cell)

Cell 10, 0.0 seconds
Cell 20, 0.0 seconds
Cell 30, 1.0 seconds
Cell 40, 1.0 seconds
Cell 50, 1.0 seconds
Cell 60, 1.0 seconds
Cell 70, 2.0 seconds
Cell 80, 2.0 seconds
Cell 90, 2.0 seconds
Cell 100, 3.0 seconds


## Lipofuscin ROIs (annotated)

In [30]:
n_rois = len(l_rois)
print('{0} rois found'.format(n_rois))
viewer.add_shapes(l_rois, name = 'Lipofuscin ROIs',
                   shape_type = 'polygon', opacity = 1, face_color = 'white', 
                   edge_color = 'blue', edge_width = 3)

28 rois found


<Shapes layer 'Lipofuscin ROIs' at 0x5b0e7888>

## Filtered images

In [67]:
# viewer.add_image(data = im_diff_npy2r, name = 'Diff npy2r', colormap = 'magenta', blending = 'additive')
# viewer.add_image(data = im_diff_tac2, name = 'Diff tac2', colormap = 'green', blending = 'additive')
# viewer.add_image(data = im_diff_car4, name = 'Diff car4', colormap = 'yellow', blending = 'additive')
# viewer.add_image(data = im_diff_vip, name = 'Diff vip', colormap = 'cyan', blending = 'additive')

# viewer.add_image(data = im_diff_npy2r[0, :, :], name = 'Diff npy2r', colormap = 'magenta', blending = 'additive')
# viewer.add_image(data = im_diff_tac2[0, :, :], name = 'Diff tac2', colormap = 'green', blending = 'additive')
# viewer.add_image(data = im_diff_car4[0, :, :], name = 'Diff car4', colormap = 'yellow', blending = 'additive')

viewer.add_image(data = im_npy2r_lipo, name = 'Blurred npy2r', colormap = 'red', blending = 'additive')
viewer.add_image(data = im_tac2_lipo, name = 'Blurred tac2', colormap = 'blue', blending = 'additive')
viewer.add_image(data = im_car4_lipo, name = 'Blurred car4', colormap = 'green', blending = 'additive')
# viewer.add_image(data = im_diff_vip[0, :, :], name = 'Diff vip', colormap = 'cyan', blending = 'additive')

<Image layer 'Blurred Vip' at 0x1870059c188>

### For lipo exclusion

In [91]:
viewer.add_image(data = im_diff_car4_lipo, name = 'Blurred car4', colormap = 'green', blending = 'additive')

<Image layer 'Blurred Vip [1]' at 0x183ba148>

In [90]:
viewer.add_image(data = im_diff_npy2r_lipo, name = 'Blurred npy2r', colormap = 'red', blending = 'additive')
viewer.add_image(data = im_diff_car4_lipo, name = 'Blurred car4', colormap = 'green', blending = 'additive')
viewer.add_image(data = im_diff_tac2_lipo, name = 'Blurred tac2', colormap = 'blue', blending = 'additive')


KeyboardInterrupt: 

## Binarized images

In [83]:
#viewer.add_image(data = im_bin_npy2r, name = 'Binary npy2r', colormap = 'red', blending = 'additive')
#viewer.add_image(data = im_bin_tac2, name = 'Binary tac2', colormap = 'blue', blending = 'additive')
viewer.add_image(data = im_bin_car4, name = 'Binary car4', colormap = 'green', blending = 'additive')

<Image layer 'Binary Vip' at 0x1b091f48>

### For lipofuscin detection

In [179]:
viewer.add_image(data = im_bin_car4_lipo, name = 'Binary car4 lipo', colormap = 'green', blending = 'additive')

<Image layer 'Binary Vip lipo' at 0x509c1fc8>

In [178]:
viewer.add_image(data = im_bin_npy2r_lipo, name = 'Binary npy2r_lipo', colormap = 'red', blending = 'additive')
viewer.add_image(data = im_bin_tac2_lipo, name = 'Binary tac2 lipo', colormap = 'blue', blending = 'additive')
viewer.add_image(data = im_bin_car4_lipo, name = 'Binary car4 lipo', colormap = 'green', blending = 'additive')

KeyboardInterrupt: 

## Detected lipofuscin pixels

In [47]:
viewer.add_image(data = im_array_lipo, name = 'Detected lipofuscin', colormap = 'gray', blending = 'additive')

C:\Users\JRCLUST\Anaconda3\lib\site-packages\scipy\ndimage\interpolation.py:611: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


<Image layer 'Detected lipofuscin' at 0x198a00f1e08>

## Cell masks after removing lipofuscin (check)

In [165]:
im_array_cells = np.zeros(im_array_npy2r.shape)


# indices = list(Cell.cell_data.keys())
indices = cells_sub + 1
for cell in indices:
    if np.mod(cell, 10) == 0:
        print('Cell {0}'.format(cell))
    planes = Cell.cell_data[cell]['z_planes']
    for plane in planes:
        mask = cell_pixels_no_lipo[cell][plane]
        mask = np.concatenate((np.ones([1, mask.shape[1]])*plane, mask))
        mask = mask.astype(int)
        im_array_cells[mask[0, :], mask[1, :], mask[2, :]] = np.ones(mask.shape[1])
        

viewer.add_image(data = im_array_cells, name = 'Cell pixels', colormap = 'blue', blending = 'additive')

Cell 10
Cell 20
Cell 30
Cell 40
Cell 50
Cell 60
Cell 80
Cell 90
Cell 120


<Image layer 'Cell pixels' at 0x1af217c8>

In [163]:
mask.shape

(4153, 2)

## Cells with significant expression

In [ ]:
signal = int_npy2r - avg_bg_npy2r
order = np.argsort(signal)
npos = pv_npy2r3 < 0.05
#indices = order[np.logical_not(npos[order])]
indices = order[npos[order]]
indices = indices[:50]

In [ ]:
indices

In [86]:
# indices = [22]
npy2r_mask_layer = viewer.add_shapes(name = 'npy2r+ {0}'.format(indices[0]))

indices = np.array(list(Cell.cell_data.keys())) - 1


for cell in indices:
    
    if pv_npy2r3[cell] < 0.05:
        n_sig = int_npy2r_bin - avg_bg_npy2r_bin
        #if np.logical_and(n_sig[cell - 1] > 150, n_sig[cell - 1] < 250):
        #if np.logical_and(n_sig[cell - 1] > 100, n_sig[cell - 1] < 150):
        planes = Cell.cell_data[cell + 1]['z_planes']
        for plane in planes:
            mask = Cell.cell_data[cell + 1]['masks'][plane]
            mask = np.concatenate((np.ones([mask.shape[0], 1])*plane, mask), axis = 1)
            npy2r_mask_layer.add(mask, shape_type = 'polygon', opacity = 0.2, face_color = 'white', edge_color = 'red', edge_width = 3)

In [87]:
# indices = [22]
car4_mask_layer = viewer.add_shapes(name = 'car4+ {0}'.format(indices[0]))

indices = np.array(list(Cell.cell_data.keys())) - 1


for cell in indices:
    
    if pv_car43[cell] < 0.05:
        n_sig = int_car4_bin - avg_bg_car4_bin
        #if np.logical_and(n_sig[cell - 1] > 150, n_sig[cell - 1] < 250):
        #if np.logical_and(n_sig[cell - 1] > 100, n_sig[cell - 1] < 150):
        planes = Cell.cell_data[cell + 1]['z_planes']
        for plane in planes:
            mask = Cell.cell_data[cell + 1]['masks'][plane]
            mask = np.concatenate((np.ones([mask.shape[0], 1])*plane, mask), axis = 1)
            npy2r_mask_layer.add(mask, shape_type = 'polygon', opacity = 0.2, face_color = 'white', edge_color = 'red', edge_width = 3)

In [88]:
# indices = [22]
tac2_mask_layer = viewer.add_shapes(name = 'tac2+ {0}'.format(indices[0]))

indices = np.array(list(Cell.cell_data.keys())) - 1


for cell in indices:
    
    if pv_tac23[cell] < 0.05:
        n_sig = int_npy2r_bin - avg_bg_npy2r_bin
        #if np.logical_and(n_sig[cell - 1] > 150, n_sig[cell - 1] < 250):
        #if np.logical_and(n_sig[cell - 1] > 100, n_sig[cell - 1] < 150):
        planes = Cell.cell_data[cell + 1]['z_planes']
        for plane in planes:
            mask = Cell.cell_data[cell + 1]['masks'][plane]
            mask = np.concatenate((np.ones([mask.shape[0], 1])*plane, mask), axis = 1)
            npy2r_mask_layer.add(mask, shape_type = 'polygon', opacity = 0.2, face_color = 'white', edge_color = 'red', edge_width = 3)

In [89]:
pv_tac23

array([1.    , 1.    , 1.    , 1.    , 1.    , 1.    , 1.    , 1.    ,
       1.    , 1.    , 1.    , 1.    , 1.    , 1.    , 1.    , 1.    ,
       0.    , 1.    , 0.    , 1.    , 1.    , 1.    , 1.    , 1.    ,
       1.    , 1.    , 1.    , 1.    , 1.    , 1.    , 1.    , 1.    ,
       1.    , 1.    , 1.    , 0.    , 1.    , 1.    , 1.    , 1.    ,
       1.    , 1.    , 1.    , 1.    , 1.    , 0.    , 1.    , 1.    ,
       1.    , 1.    , 1.    , 1.    , 0.5   , 0.5   , 1.    , 1.    ,
       0.4945, 1.    , 1.    , 1.    , 0.5   , 1.    , 1.    , 0.001 ,
       1.    , 1.    , 0.989 , 0.    , 1.    , 1.    , 1.    , 0.    ,
       1.    , 0.    , 0.    , 1.    , 1.    , 1.    , 1.    , 1.    ,
       0.    , 0.5   , 1.    , 1.    , 1.    , 1.    , 1.    , 0.    ,
       0.    , 1.    , 1.    , 1.    , 1.    , 1.    , 1.    , 1.    ,
       1.    , 0.    , 1.    , 1.    ])

# Cell segmentation

## Load existing masks

In [5]:
save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR 3.12'
save_file= 'S3_data.pkl'
try:
    with open('{0}\{1}'.format(save_loc, save_file), 'rb') as f:
        Cell.cell_data = pkl.load(f)
        indices = list(Cell.cell_data.keys())
        if not np.max(indices) == len(indices):
            print('Re-numbering cells to be consecutive')
            Cell.cell_data_temp = {}
            for i in range(len(indices)):
                Cell.cell_data_temp[i + 1] = Cell.cell_data[indices[i]]
                Cell.cell_data_temp[i + 1]['cell_id'] = i + 1
            Cell.cell_data = Cell.cell_data_temp 
            with open('{0}\{1}'.format(save_loc, save_file), 'wb') as f:
                pkl.dump(Cell.cell_data, f)
            Cell.n_cells = i + 1
        else:
            Cell.n_cells = len(indices)
    print('{0} cells found'.format(Cell.n_cells))
except:
    print('No data found')

101 cells found


## If loading a subset of planes, subselect cells

In [21]:
list(plane_nos)

[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [29]:
plt.figure()
plt.plot(all_cell_pixels[:, 0])

In [6]:
cells_sub = []
for cell in range(Cell.n_cells):
    flag = 0
    cell_no = cell + 1
    cp = Cell.cell_data[cell_no]['z_planes']
    for p in cp:
        if p + 1 not in list(plane_nos):
            flag = 1
    if flag == 0:
        cells_sub = np.append(cells_sub, cell)

cells_sub = np.array(cells_sub)
cells_sub = cells_sub.astype(int)
print(cells_sub)

[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  98  99 100]


In [23]:
len(cells_sub)

100

In [24]:
Cell.n_cells

101

In [26]:
[i for i in list(Cell.cell_data.keys()) if i - 1 not in cells_sub]

[98]

In [27]:
Cell.cell_data.pop(98)

{'cell_id': 98,
 'z_planes': [7.0, 8.0, 9.0],
 'masks': {7.0: array([[8828.27565361, 4778.41385369],
         [8838.8962664 , 4791.09710966],
         [8828.46957543, 4800.24808697],
         [8814.71156237, 4803.83117073],
         [8804.58155986, 4805.85168767],
         [8796.26472371, 4806.68623911],
         [8789.03397543, 4806.35680347],
         [8782.79154305, 4806.37278928],
         [8776.74575771, 4806.04499632],
         [8771.28192342, 4803.49622512],
         [8767.24597698, 4799.10607864],
         [8763.12576631, 4795.60634016],
         [8758.14050986, 4792.33380783],
         [8753.79371165, 4787.67359806],
         [8749.46323898, 4781.87366228],
         [8747.47496307, 4774.73780706],
         [8747.16013945, 4766.9189986 ],
         [8746.14450939, 4757.27614554],
         [8750.05784746, 4748.84277696],
         [8759.38935806, 4746.13398276],
         [8770.17150895, 4750.54455209],
         [8777.0693217 , 4752.7563641 ],
         [8782.62912633, 4753.45051834

## Get pixels in cell masks

In [28]:
save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR 3.12'
#save_file = 'S2_cell_pixels.pkl'
save_file = 'S3_mask_vertices.pkl'

t0 = time.time()

try:
    with open('{0}\\{1}'.format(save_loc, save_file), 'rb') as f:
        mask_vertices = pkl.load(f)
        cells = Cell.cell_data.keys()
#         cells = cells_sub + 1
        x = 0
        all_cell_pixels = np.zeros([1, 3])
        for cell in cells:
            data = Cell.cell_data[cell]
            planes = data['z_planes']
            for plane in planes:
                xvals = np.reshape(mask_vertices[cell][plane][0], [-1, 1])
                yvals = np.reshape(mask_vertices[cell][plane][1], [-1, 1])
                zvals = np.ones([len(xvals), 1])*plane
                coords = np.concatenate((zvals, xvals, yvals), axis = 1)
                all_cell_pixels = np.concatenate((all_cell_pixels, coords), axis = 0)
        
        all_cell_pixels = all_cell_pixels[1:, :].astype(int)
            
        print('Data loaded')
        
except IOError:
    print('No saved data found, calculating mask pixels')
#     xv = range(w)
#     yv = range(h)
#     coord_array = np.array(np.meshgrid(xv, yv))

#     points = np.zeros([h*w, 2])
#     p = 0
#     for i in range(h):
#         for j in range(w):
#             points[p, 1] = coord_array[0, i, j]
#             points[p, 0] = coord_array[1, i, j]
#             p += 1

    no_cells = len(Cell.cell_data.keys())
    mask_vertices = {}
    for cell in range(no_cells):
        if np.mod(cell, 2) == 0:
            print('Cell {0}: {1} seconds'.format(cell, np.round(time.time() - t0)))
        cell_no = cell + 1
        mask_vertices[cell_no] = {}
        cell_dict = Cell.cell_data[cell_no]
        masks = cell_dict['masks']
        z_planes = cell_dict['z_planes']
        for plane in z_planes:

            vertices = masks[plane]
            path = mpltpath.Path(vertices)
            mask = path.contains_points(points)
            mask = np.reshape(mask, [h, w])
            mask_vertices[cell_no][plane] = np.where(mask)
    
    with open('{0}\\{1}'.format(save_loc, save_file), 'wb') as f:
        pkl.dump(mask_vertices, f)

Data loaded


In [35]:
with open('{0}\\{1}'.format(save_loc, save_file), 'wb') as f:
    pkl.dump(mask_vertices, f)

## Semi-manual segmentation using vip channel

### Data structure to store cell masks

In [5]:
class Cell:
    n_cells = 1
    cell_data = {}
    
    def __init__(self, cell_no):
        
        self.cell_no = cell_no
        self.z_planes = []
        self.no_planes = 0
        self.masks = {}
        self.c_vals = np.zeros(1)
        print('Cell #{0} created'.format(Cell.n_cells))
        
    def add_mask(self, z_plane, mask):
        self.z_planes.append(z_plane)
        self.no_planes += 1
        self.masks[z_plane] = mask
            
    def save_cell(self):
        Cell.cell_data[self.cell_no] = {'cell_id': self.cell_no,
                                 'z_planes': self.z_planes,
                                 'masks': self.masks,
                                 'no_planes': self.no_planes,
                                 'c_vals':self.c_vals}

### Napari keybindings

In [35]:
mask_layer = viewer.add_shapes(data = None, shape_type = 'polygon', opacity = 0.2, face_color = 'white', edge_color = 'red', edge_width = 3)
point_layer = viewer.add_points(data = None)
save_loc = 'G:{0}Shared drives{0}as_share{0}HCR{0}HCR 3.11'.format(sep)
save_file = 'S3_data.pkl'

@viewer.bind_key('n', overwrite = True)
def new_cell(viewer):
    global cell1 
    cell1 = Cell(Cell.n_cells)
    print(Cell.n_cells)
    Cell.n_cells += 1
    
    
@viewer.bind_key('d', overwrite = True)
def get_mask(viewer):
    center = point_layer.data[-1, 1:] # Last drawn point
    z_plane = point_layer.data[-1, 0].astype(int)
    image = im_array_npy2r_blur[z_plane, :, :]
    cell_radius_px = 70

    boundary = disc_roi(image, center, z_plane, cell_radius_px)

    mask_layer.add(boundary, shape_type= 'polygon')
    
    
@viewer.bind_key('m', overwrite = True)
def add_mask(viewer):
    
    z_plane = mask_layer.data[-1][0, 0]
    mask = mask_layer.data[-1][:, 1:]
    cell1.add_mask(z_plane, mask)
    
@viewer.bind_key('y', overwrite = True)
def del_cell(viewer):
    
    mask_no = mask_layer2.selected_data[0]
    cell = mask_cell_nos[mask_no]
    z_plane = mask_layer2.coordinates[0]
    
    Cell.cell_data[cell]['masks'][z_plane] = mask_layer2.data[mask_no][:, 1:]
    with open('{0}\{1}'.format(save_loc, save_file), 'wb') as f:
                pkl.dump(Cell.cell_data, f)
    
    mask = Cell.cell_data[cell]['masks'][z_plane]
    n_vertices = mask.shape[0]
    mask = np.concatenate([np.ones([n_vertices, 1])*z_plane, mask], axis = 1)
    mask_layer2.add(mask, shape_type = 'polygon', opacity = 0.4, 
                    face_color = 'white', edge_color = 'yellow', edge_width = 5)
    
    
@viewer.bind_key('k', overwrite = True)
def save_cell(viewer):
    cell1.save_cell()
    
@viewer.bind_key('Control-s', overwrite = True)
def save_all(viewer):
    with open('{0}\{1}'.format(save_loc, save_file), 'wb') as f:
                pkl.dump(Cell.cell_data, f)

In [12]:
del_cell(viewer)

IndexError: list index out of range

### One click segmentation

#### View saved masks

In [161]:
Cell.n_cells

357

In [68]:
save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR 3.12'
save_file= 'S3_data.pkl'

with open('{0}\{1}'.format(save_loc, save_file), 'wb') as f:
        pkl.dump(Cell.cell_data, f)

In [13]:
# Check number of masks per cell, and variance of mask centers
indices = list(Cell.cell_data.keys())
n_planes = np.zeros(len(indices)).astype(int)
var_masks = np.zeros([len(indices), 2])

if not np.max(indices) == len(indices):
    print('{0} cells not recorded'.format(np.max(indices) - len(indices)))
    print('Re-numbering cells to be consecutive')
    Cell.cell_data_temp = {}
    for i in range(len(indices)):
        Cell.cell_data_temp[i + 1] = Cell.cell_data[indices[i]]
        Cell.cell_data_temp[i + 1]['cell_id'] = i + 1
    Cell.cell_data = Cell.cell_data_temp 
    with open('{0}\{1}'.format(save_loc, save_file), 'wb') as f:
        pkl.dump(Cell.cell_data, f)
        
Cell.n_cells = len(indices)
        
for i in range(len(indices)):
    z_planes = Cell.cell_data[i + 1]['z_planes']
    n_planes[i] = len(z_planes)
    masks = Cell.cell_data[i + 1]['masks']
    centers = np.zeros([n_planes[i], 2])
    for j in range(n_planes[i]):
        centers[j, 0] = np.mean(masks[z_planes[j]][:, 0])
        centers[j, 1] = np.mean(masks[z_planes[j]][:, 1])
    var_masks[i, 0] = np.var(centers[:, 0])
    var_masks[i, 1] = np.var(centers[:, 1])
    
    

In [14]:
fig, ax = plt.subplots(nrows = 2, ncols = 1, sharex = True, figsize = [15, 5])
ax[0].plot(n_planes)
ax[0].set_xlabel('Cell number')
ax[0].set_ylabel('Number of masks')
ax[1].plot(var_masks[:, 0], label = 'y_values')
ax[1].plot(var_masks[:, 1], label = 'x_values')
ax[1].set_ylabel('Variance of centers')
ax[1].plot(range(len(indices)), np.ones(len(indices))*8, '--', color = 'k')
ax[1].legend()

#### Functions

In [36]:
def disc_roi(image, center, z_plane, cell_radius_px, n_theta = 30, thresh = 0.5):
    
    full_radius = round(1.3*cell_radius_px)
    thetas = np.linspace(0, 2*np.pi, n_theta)
    [x, y] = center
    
    
    line_profiles = np.zeros([full_radius, n_theta])
    
    r_threshold_cross = np.zeros(n_theta)
    
    for t in range(n_theta):
        theta = thetas[t]
        f = profile_line(image, (x, y), (x + full_radius*np.cos(theta), y + full_radius*np.sin(theta)))
        r_threshold_cross[t] = find_first_cross(f, thresh)
        line_profiles[:, t] = -1*np.abs(np.array(range(full_radius)) - r_threshold_cross[t]) + full_radius;
        
    #path = find_path(line_profiles)
   
    #boundary = np.zeros([2, n_theta])
    #boundary[0, :] = x + np.multiply(path, np.cos(thetas))
    #boundary[1, :] = y + np.multiply(path, np.sin(thetas))
    #boundary = boundary.astype(int)
    
    angular_dist = np.argmax(line_profiles, axis = 0)
    boundary = np.zeros([n_theta, 3])
    boundary[:, 0] = np.ones(n_theta)*z_plane
    boundary[:, 1] = x + np.multiply(angular_dist, np.cos(thetas))
    boundary[:, 2] = y + np.multiply(angular_dist, np.sin(thetas))
    
    
    
    return boundary

def find_first_cross(line_profile, thresh):
    
    # Assumes line profile goes from inside cell to outside
    max_val = np.max(line_profile)
    max_idx = np.argmax(line_profile)
    min_val_out = np.min(line_profile[max_idx:]) # Minimum intensity outside cell
    
    thresh_val = min_val_out + thresh*(max_val - min_val_out)
    r_cross = max_idx + np.where(line_profile[max_idx:] < thresh_val)[0][0]
    
    return r_cross

def find_path(line_profiles):
    
    # line_profiles should be of shape radius X n_theta
    radius = line_profiles.shape[0]
    n_theta = line_profiles.shape[1]
    
    pointer = np.zeros([radius, n_theta])
    value = np.zeros([radius, n_theta])
    value[:, 0] = line_profiles[:, 0]
    
    for i in range(1, n_theta):
        for j in range(1, radius - 1):
            
            M = np.max(value[j-1:j+1, i-1])
            ind = np.argmax(value[j-1:j+1, i-1])
            
            value[j, i] = M + line_profiles[j, i]
            pointer[j,i] = j + ind - 1
            
    # Second traverse to minimize boundary effect
    pointer = np.zeros([radius, n_theta])
    value[:, 1] = value[:, -1]
    
    for i in range(1, n_theta):
        for j in range(1, radius - 1):
            
            M = np.max(value[j-1:j+1, i-1])
            ind = np.argmax(value[j-1:j+1, i-1])
            
            value[j, i] = M + line_profiles[j, i]
            pointer[j,i] = j + ind - 1
    
    path = np.zeros(n_theta).astype(int)
    
    M = np.max(value[:, -1])
    ind = np.argmax(value[:, -1])
    path[-1] = ind
    
    for j in np.flip(range(1, n_theta)):
        path[j-1] = pointer[path[j], j]
        
    return path

#### Find intensity profile at a line joining two points

In [77]:
point_layer = viewer.add_points(data = None)

In [85]:
image = im_array_vip_blur[0, :, :]

[x1, y1] = point_layer.data[0, 1:]
[x2, y2] = point_layer.data[1, 1:]
z = point_layer.data[0, 0]

profile = profile_line(image, (x1, y1), (x2, y2))
n_points = profile.shape
px_dist = np.floor(np.linalg.norm(np.array([x1, y1]) - np.array([x2, y2])))
print('Pixel distance = {0}, number of points in profile = {1}'.format(px_dist, n_points))

print('x1 = {0}, y1 = {1}, x2 = {2}, y2 = {3}'.format(x1, y1, x2, y2))

Pixel distance = 58.0, number of points in profile = (60,)
x1 = 1124.5762199044127, y1 = 10278.852064061517, x2 = 1083.8527461351187, y2 = 10320.211842108456


#### Find point where intensity falls below threshold

In [86]:
thresh = 0.1
line_profile = profile
max_val = np.max(line_profile)
max_idx = np.argmax(line_profile)
min_val_out = np.min(line_profile[max_idx:]) # Minimum intensity outside cell
thresh_val = min_val_out + thresh*(max_val - min_val_out)
r_cross = max_idx + np.where(line_profile[max_idx:] < thresh_val)[0][0]

In [87]:
plt.figure()
plt.plot(line_profile, color = 'k')

xl = np.array(plt.gca().get_xlim())
yl = np.array(plt.gca().get_ylim())
xi = np.linspace(xl[0], xl[1], 10)
yi = np.linspace(yl[0], yl[1], 10)

plt.plot(xi, np.ones(10)*max_val, color = 'r', linestyle = '--')
plt.plot(xi, np.ones(10)*min_val_out, color = 'r', linestyle = '--')
plt.plot(xi, np.ones(10)*thresh_val, color = 'b', linestyle = '--')
plt.plot(np.ones(10)*r_cross, yi, color = 'b', linestyle = '--')

#### Find intensity profile at several angles

In [92]:
center = [x1, y1]
cell_radius_px = 70
n_theta = 30
thresh = 0.2

full_radius = round(1.3*cell_radius_px)
thetas = np.linspace(0, 2*np.pi, n_theta)
[h, w] = image.shape
[x, y] = center


line_profiles = np.zeros([full_radius, n_theta])
line_profiles_mod = np.zeros([full_radius, n_theta])

r_threshold_cross = np.zeros(n_theta)

for t in range(n_theta):
    theta = thetas[t]
    f = profile_line(image, (x, y), (x + full_radius*np.cos(theta), y + full_radius*np.sin(theta)))
    line_profiles[:, t] = f[:full_radius]
    r_threshold_cross[t] = find_first_cross(f, thresh)
    line_profiles_mod[:, t] = -1*np.abs(np.array(range(full_radius)) - r_threshold_cross[t]) + full_radius;
    
plt.figure()
plt.imshow(line_profiles_mod)
plt.xlabel('Theta/4')
plt.ylabel('Distance from center (px)')
plt.title('Pixel intensity')

Text(0.5, 1.0, 'Pixel intensity')

In [67]:
fig, ax = plt.subplots(nrows = 1, ncols = 2)
ax[0].imshow(line_profiles)
ax[1].imshow(line_profiles_mod)

In [93]:
angular_dist = np.argmax(line_profiles_mod, axis = 0)
boundary = np.zeros([n_theta, 3])
boundary[:, 0] = np.ones(n_theta)*z
boundary[:, 1] = x + np.multiply(angular_dist, np.cos(thetas))
boundary[:, 2] = y + np.multiply(angular_dist, np.sin(thetas))

#### Find boundary of cell from intensity profiles

In [39]:
radius = line_profiles.shape[0]
n_theta = line_profiles.shape[1]

pointer = np.zeros([radius, n_theta])
value = np.zeros([radius, n_theta])
value[:, 0] = line_profiles_mod[:, 0]

for i in range(1, n_theta):
    for j in range(1, radius - 1):

        M = np.max(value[j-1:j+1, i-1])
        ind = np.argmax(value[j-1:j+1, i-1])

        value[j, i] = M + line_profiles_mod[j, i]
        pointer[j,i] = j + ind - 1
        
fig, ax = plt.subplots(nrows = 2, ncols = 2, figsize = (20, 10))
ax[0, 0].imshow(value)
ax[0, 0].set_xlabel('Theta/4')
ax[0, 0].set_ylabel('Distance from center (px)')
ax[0, 0].set_title('Value - first iteration')

ax[0, 1].imshow(pointer)
ax[0, 1].set_xlabel('Theta/4')
ax[0, 1].set_ylabel('Distance from center (px)')
ax[0, 1].set_title('Pointer - first iteration')

# Second traverse to minimize boundary effect
pointer = np.zeros([radius, n_theta])
value[:, 1] = value[:, -1]

for i in range(1, n_theta):
    for j in range(1, radius - 1):

        M = np.max(value[j-1:j+1, i-1])
        ind = np.argmax(value[j-1:j+1, i-1])

        value[j, i] = M + line_profiles[j, i]
        pointer[j,i] = j + ind - 1

ax[1, 0].imshow(value)
ax[1, 0].set_xlabel('Theta/4')
ax[1, 0].set_ylabel('Distance from center (px)')
ax[1, 0].set_title('Value - second iteration')

ax[1, 1].imshow(pointer)
ax[1, 1].set_xlabel('Theta/4')
ax[1, 1].set_ylabel('Distance from center (px)')
ax[1, 1].set_title('Pointer - second iteration')

path = np.zeros(n_theta).astype(int)
    
M = np.max(value[:, -1])
ind = np.argmax(value[:, -1])
path[-1] = ind

for j in np.flip(range(1, n_theta)):
    path[j-1] = pointer[path[j], j]
    
boundary = np.zeros([n_theta, 3])
boundary[:, 0] = np.ones(n_theta)*z
boundary[:, 1] = x + np.multiply(path, np.cos(thetas))
boundary[:, 2] = y + np.multiply(path, np.sin(thetas))

# boundary = np.zeros([2, n_theta])
# boundary[0, :] = x + np.multiply(path, np.cos(thetas))
# boundary[1, :] = y + np.multiply(path, np.sin(thetas))

boundary = boundary.astype(int)
    
ax[1, 1].plot(path, color = 'r', linestyle = '--')

# Lipofuscin exlcusion

## Load existing lipofuscin ROIs

In [5]:
save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR 3.12\\S3'
save_file = 'HCR_3.12_S3_lipofuscin_rois_in_cells.pkl'

with open('{0}\\{1}'.format(save_loc, save_file), 'rb') as f:
    l_rois = pkl.load(f)
    print('{0} rois'.format(len(l_rois)))

35 rois


## Save lipofuscin rois from napari viewer

In [15]:
viewer.add_shapes(data = None, name = 'Lipofuscin ROIs')

<Shapes layer 'Lipofuscin ROIs' at 0x1987fc45a08>

In [14]:
viewer.layers

[<Image layer 'Ndnf' at 0x4bbac148>,
 <Image layer 'Vip' at 0x4e2edf48>,
 <Image layer 'Sst' at 0x509d3088>,
 <Shapes layer 'Lipofuscin ROIs ' at 0x50d64b08>]

In [16]:
save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR 3.12\S3'
save_file = 'HCR_3.12_S3_lipofuscin_rois_in_cells.pkl'

l_rois = viewer.layers['Lipofuscin ROIs'].data
with open('{0}\\{1}'.format(save_loc, save_file), 'wb') as f:
    pkl.dump(l_rois, f) 
    
n_rois = len(l_rois)
print('{0} lipofuscin rois'.format(n_rois))

11 lipofuscin rois


## Get pixels in annotated lipofuscin ROIs

In [17]:
save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR 3.12\S3'
save_file = 'HCR_3.12_S3_lipofuscin_pixels_in_cells.pkl'
    
try:
    with open('{0}{1}d{2}'.format(save_loc, sep, save_file), 'rb') as f:
        data_dict = pkl.load(f)
        all_lipo_pixels = data_dict['all_lipo_pixels']
        lipo_pixels_roi = data_dict['lipo_pixels_roi']
        print('Data loaded')
        
        
except IOError:
    print('No saved data found, calculating mask pixels')
    px = pixels_in_roi.pixels_in_roi(h, w, n, l_rois)
    all_lipo_pixels = px['all_pixels']
    lipo_pixels_roi = px['pixels_roi']
    with open('{0}{1}{2}'.format(save_loc, sep, save_file), 'wb') as f:
        pkl.dump({'all_lipo_pixels': all_lipo_pixels, 'lipo_pixels_roi': lipo_pixels_roi}, f)

No saved data found, calculating mask pixels
Calculated grid coordinates, 77.0 seconds
ROI 0
ROI 1
ROI 2
ROI 3
ROI 4
ROI 5
ROI 6
ROI 7
ROI 8
ROI 9
ROI 10


## Filter images using difference of gaussians

### Diff_gauss2

In [18]:
from scipy.stats import norm
from skimage.filters import gaussian

def diff_gauss2(sigma_small, sigma_large, img, do_plot = 1):

    #sigma_large = 10 # In pixels
    #sigma_small = 3 # In pixels

    #x = np.linspace(norm.ppf(0.01), norm.ppf(0.99), 1000)
    x = np.linspace(-sigma_large*5, sigma_large*5, 1000)
    small = norm.pdf(x, scale = sigma_small)
    large = norm.pdf(x, scale = sigma_large)
    diff = small - large

    n = img.shape[0]
    im_diff = np.zeros(img.shape)

    for plane in range(n):
        im_filt_large = gaussian(img[plane, :, :], sigma = sigma_large)
        im_filt_small = gaussian(img[plane, :, :], sigma = sigma_small)
        im_diff[plane, :, :] = im_filt_small - im_filt_large

    if do_plot:

        plt.figure()
        plt.plot(x, diff, label = 'diff')
        plt.plot(x, small, label = 'Small sigma filter')
        plt.plot(x, large, label = 'Large sigma filter')
        plt.xlabel('Distance from center of kernel (px)')
        plt.ylabel('Kernel')
        plt.title('Difference of gaussians filter')
        plt.legend()

    return im_diff


### Do filtering

#### Diff_gauss2

In [19]:
t0 = time.time()    

print('Filtering images (~5 minutes)')
sigma_small = 5
sigma_large = 10


img = im_array_npy2r
im_diff_npy2r_lipo = diff_gauss2(sigma_small, sigma_large, img, do_plot = 1)
t1 = time.time() - t0
print('{0} seconds'.format(int(t1)))

img = im_array_tac2
im_diff_tac2_lipo = diff_gauss2(sigma_small, sigma_large, img, do_plot = 0)
t1 = time.time() - t0
print('{0} seconds'.format(int(t1)))

img = im_array_car4
im_diff_car4_lipo = diff_gauss2(sigma_small, sigma_large, img, do_plot = 0)
t1 = time.time() - t0
print('{0} seconds'.format(int(t1)))

# img = im_array_vip
# im_diff_vip_lipo = diff_gauss.diff_gauss(sigma_small, sigma_large, img, do_plot = 1)
# t1 = time.time() - t0
# print('{0} seconds'.format(int(t1)))

del img

Filtering images (~5 minutes)
182 seconds
365 seconds
549 seconds


#### Simple gaussian filter

In [66]:
t0 = time.time()    

print('Filtering images (~5 minutes)')
sigma = 3


img = im_array_npy2r
im_npy2r_lipo = gaussian(img, sigma)
t1 = time.time() - t0
print('{0} seconds'.format(int(t1)))

img = im_array_tac2
im_tac2_lipo = gaussian(img, sigma)
t1 = time.time() - t0
print('{0} seconds'.format(int(t1)))

img = im_array_car4
im_car4_lipo = gaussian(img, sigma)
t1 = time.time() - t0
print('{0} seconds'.format(int(t1)))

# img = im_array_vip
# im_diff_vip_lipo = diff_gauss.diff_gauss(sigma_small, sigma_large, img, do_plot = 1)
# t1 = time.time() - t0
# print('{0} seconds'.format(int(t1)))

del img

Filtering images (~5 minutes)
33 seconds
65 seconds
97 seconds


In [36]:
t0 = time.time()    

try:
    jhui
    save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR 3.12\\S3'

    save_file = 'vip_im_diff_sigma_small = {0}, sigma_large = {1}.pkl'.format(sigma_small, sigma_large)
    h5f = h5py.File('{0}{1}{2}.h5'.format(save_loc, sep, save_file),'r')
    im_diff_vip = h5f['dataset_1'][:]
    h5f.close()
    print('Loaded im_diff_vip, {0} seconds'.format(np.round(time.time() - t0)))
    
    save_file = 'npy2r_im_diff_sigma_small = {0}, sigma_large = {1}.pkl'.format(sigma_small, sigma_large)
    h5f = h5py.File('{0}{1}{2}.h5'.format(save_loc, sep, save_file),'r')
    im_diff_npy2r = h5f['dataset_1'][:]
    h5f.close()
    print('Loaded im_diff_npy2r, {0} seconds'.format(np.round(time.time() - t0)))
    
    save_file = 'tac2_im_diff_sigma_small = {0}, sigma_large = {1}.pkl'.format(sigma_small, sigma_large)
    h5f = h5py.File('{0}{1}{2}.h5'.format(save_loc, sep, save_file),'r')
    im_diff_tac2 = h5f['dataset_1'][:]
    h5f.close()
    print('Loaded im_diff_tac2, {0} seconds'.format(np.round(time.time() - t0)))
    
    save_file = 'car4_im_diff_sigma_small = {0}, sigma_large = {1}.pkl'.format(sigma_small, sigma_large)
    h5f = h5py.File('{0}{1}{2}.h5'.format(save_loc, sep, save_file),'r')
    im_diff_car4 = h5f['dataset_1'][:]
    h5f.close()
    print('Loaded im_diff_car4, {0} seconds'.format(np.round(time.time() - t0)))
    

except:
    print('No images found, filtering images (~5 minutes)')
    sigma_small = 1
    sigma_large = 2


    img = im_array_npy2r
    im_diff_npy2r = diff_gauss.diff_gauss(sigma_small, sigma_large, img, do_plot = 1)
    t1 = time.time() - t0
    print('{0} seconds'.format(int(t1)))

    img = im_array_tac2
    im_diff_tac2 = diff_gauss.diff_gauss(sigma_small, sigma_large, img, do_plot = 0)
    t1 = time.time() - t0
    print('{0} seconds'.format(int(t1)))

    img = im_array_car4
    im_diff_car4 = diff_gauss.diff_gauss(sigma_small, sigma_large, img, do_plot = 0)
    t1 = time.time() - t0
    print('{0} seconds'.format(int(t1)))

#     img = im_array_vip
#     im_diff_vip = diff_gauss.diff_gauss(sigma_small, sigma_large, img, do_plot = 1)
#     t1 = time.time() - t0
#     print('{0} seconds'.format(int(t1)))

    del img

No images found, filtering images (~5 minutes)
13 seconds
27 seconds
41 seconds


In [12]:
img = im_array_vip
im_diff_vip = diff_gauss.diff_gauss(sigma_small, sigma_large, img, do_plot = 1)
t1 = time.time() - t0
print('{0} seconds'.format(int(t1)))

del img

959 seconds


In [55]:
t0 = time.time()
save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR 3.11\\S3'

save_file = 'vip_im_diff_sigma_small = {0}, sigma_large = {1}'.format(sigma_small, sigma_large)
h5f = h5py.File('{0}{1}{2}.h5'.format(save_loc, sep, save_file), 'w')
h5f.create_dataset('dataset_1', data=im_diff_vip)
h5f.close()

print('im_diff_vip saved: {0} seconds'.format(np.round(time.time() - t0)))
    
save_file = 'npy2r_im_diff_sigma_small = {0}, sigma_large = {1}'.format(sigma_small, sigma_large)
h5f = h5py.File('{0}{1}{2}.h5'.format(save_loc, sep, save_file), 'w')
h5f.create_dataset('dataset_1', data=im_diff_npy2r)
h5f.close()

print('im_diff_npy2r saved: {0} seconds'.format(np.round(time.time() - t0)))
    
save_file = 'tac2_im_diff_sigma_small = {0}, sigma_large = {1}'.format(sigma_small, sigma_large)
h5f = h5py.File('{0}{1}{2}.h5'.format(save_loc, sep, save_file), 'w')
h5f.create_dataset('dataset_1', data=im_diff_tac2)
h5f.close()

print('im_diff_tac2 saved: {0} seconds'.format(np.round(time.time() - t0)))
    
save_file = 'car4_im_diff_sigma_small = {0}, sigma_large = {1}'.format(sigma_small, sigma_large)
h5f = h5py.File('{0}{1}{2}.h5'.format(save_loc, sep, save_file), 'w')
h5f.create_dataset('dataset_1', data=im_diff_car4)
h5f.close()

print('im_diff_car4 saved: {0} seconds'.format(np.round(time.time() - t0)))

im_diff_gad1 saved: 13.0 seconds
im_diff_ndnf saved: 27.0 seconds
im_diff_sst saved: 41.0 seconds
im_diff_vip saved: 57.0 seconds


## Binarize filtered images and plot threshold

In [33]:
img_dict = {'npy2r': im_diff_npy2r_lipo, 'car4': im_diff_car4_lipo, 'tac2': im_diff_tac2_lipo}
# img_dict = {'npy2r': im_array_npy2r, 'tac2': im_array_tac2, 'car4': im_array_car4}

pixels_dict = {'Cells': all_cell_pixels, 'Lipofuscin': all_lipo_pixels}
#pixels_dict = {'Cells': all_cell_pixels}

colors_dict = {'Cells': 'b', 'Lipofuscin': 'k'}

title = 'Diff gauss sigma = ({0}, {1})'.format(sigma_small, sigma_large)
#title = 'Raw images'
save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR 3.12\\S3\\Plots'
save_file = '{0} hist lipo + cells.png'.format(title)
#save_file = '{0} hist cells.png'.format(title)

thresh_scale = 1

thresh = hist_channels.hist_channels(img_dict, pixels_dict, max_per_group= 100000, 
                                     do_bin = True, bin_group = 'Cells', thresh_scale = thresh_scale,
                                     colors_dict = colors_dict, 
                                     title = title, save = True, save_loc = save_loc, save_file = save_file)

del img_dict
del pixels_dict

im_bin_npy2r_lipo = np.zeros(im_diff_npy2r_lipo.shape)
im_bin_tac2_lipo = np.zeros(im_diff_npy2r_lipo.shape)
im_bin_car4_lipo = np.zeros(im_diff_npy2r_lipo.shape)
#im_bin_vip = np.zeros(im_diff_vip_lipo.shape)

ix_npy2r = np.where(im_diff_npy2r_lipo > thresh['npy2r'])
ix_tac2 = np.where(im_diff_tac2_lipo > thresh['tac2'])
ix_car4 = np.where(im_diff_car4_lipo > thresh['car4'])
# ix_vip = np.where(im_diff_vip_lipo > thresh['vip'])

im_bin_npy2r_lipo[ix_npy2r] = im_diff_npy2r_lipo[ix_npy2r]
im_bin_tac2_lipo[ix_tac2] = im_diff_tac2_lipo[ix_tac2]
im_bin_car4_lipo[ix_car4] = im_diff_car4_lipo[ix_car4]
#im_bin_vip_lipo[ix_vip] = im_diff_vip_lipo[ix_vip]

Finding threshold for binarizing
Finding threshold for binarizing
Finding threshold for binarizing


## Select lipofuscin cloud

### Roipoly

In [13]:
# This code is from https://github.com/jdoepfert, 
# which can be used without permission
import matplotlib.path as mplPath

class roipoly:
    def __init__(self, fig=[], ax=[], roicolor='b'):
        if fig == []:
            fig = plt.gcf()

        if ax == []:
            ax = plt.gca()

        self.previous_point = []
        self.allxpoints = []
        self.allypoints = []
        self.start_point = []
        self.end_point = []
        self.line = None
        self.roicolor = roicolor
        self.fig = fig
        self.ax = ax
        self.mask = []

        self.__ID1 = self.fig.canvas.mpl_connect(
            'motion_notify_event', self.__motion_notify_callback)
        self.__ID2 = self.fig.canvas.mpl_connect(
            'button_press_event', self.__button_press_callback)
#         self.__ID3 = self.fig.canvas.mpl_connect(
#             'key_press_event', self.__key_press_callback)

    def getMask(self):
        
        ny = 20000
        nx = 20000
        poly_verts = [(self.allxpoints[0], self.allypoints[0])]
        for i in range(len(self.allxpoints)-1, -1, -1):
            poly_verts.append((self.allxpoints[i], self.allypoints[i]))

        # Create vertex coordinates for each grid cell...
        # (<0,0> is at the top left of the grid in this system)
        x0, x1 = self.ax.get_xlim()
        y0, y1 = self.ax.get_ylim()
        x, y = np.meshgrid(np.linspace(x0, x1, nx), np.linspace(y0, y1, ny))
        x, y = x.flatten(), y.flatten()
        points = np.vstack((x,y)).T

        ROIpath = mplPath.Path(poly_verts)
        grid = ROIpath.contains_points(points).reshape((ny,nx)), 0
        return grid

    def __motion_notify_callback(self, event):
        if event.inaxes:
            ax = event.inaxes
            x, y = event.xdata, event.ydata
            # Move line around
            if (event.button == None or event.button == 1) and self.line != None: 
                self.line.set_data([self.previous_point[0], x],
                                   [self.previous_point[1], y])
                self.fig.canvas.draw()


#     def __key_press_callback(self, event):
            
        
    def __button_press_callback(self, event):
        if event.inaxes:
            x, y = event.xdata, event.ydata
            ax = event.inaxes
            # If you press the left button, single click
            if event.button == 1 and event.dblclick == False:  
                if self.line == None: # if there is no line, create a line
                    self.line = plt.Line2D([x, x],
                                           [y, y],
                                           marker='o',
                                           color=self.roicolor)
                    self.start_point = [x,y]
                    self.previous_point =  self.start_point
                    self.allxpoints=[x]
                    self.allypoints=[y]

                    ax.add_line(self.line)
                    self.fig.canvas.draw()
                    # add a segment
                else: # if there is a line, create a segment
                    self.line = plt.Line2D([self.previous_point[0], x],
                                           [self.previous_point[1], y],
                                           marker = 'o',
                                           color=self.roicolor)
                    self.previous_point = [x,y]
                    self.allxpoints.append(x)
                    self.allypoints.append(y)

                    event.inaxes.add_line(self.line)
                    self.fig.canvas.draw()
            elif ((event.button == 1 and event.dblclick==True) or (event.button == 3 and event.dblclick==False)) and self.line != None: # close the loop and disconnect
                    
                    self.fig.canvas.mpl_disconnect(self.__ID1) #joerg
                    self.fig.canvas.mpl_disconnect(self.__ID2) #joerg

                    self.line.set_data([self.previous_point[0],
                                    self.start_point[0]],
                                   [self.previous_point[1],
                                    self.start_point[1]])
                    self.last_line = ax.add_line(self.line)
                    self.fig.canvas.draw()
                    self.line = None

                    if sys.flags.interactive:
                        pass
                    


### Scatter plot to draw on

In [14]:
fig = plt.figure()
x_vals = [im_bin_npy2r_lipo[a[0], a[1], a[2]] for a in all_lipo_pixels]
y_vals = [im_bin_car4_lipo[a[0], a[1], a[2]] for a in all_lipo_pixels]

plt.scatter(x_vals, y_vals)
roi = roipoly()

### Get pixels

In [15]:
t0 = time.time()
mask = roi.getMask()[0]
t1 = time.time() - t0
print(np.round(t1))

50.0


In [18]:
n_lipo_px = len(all_lipo_pixels)
xi = np.zeros(n_lipo_px, dtype = int)
yi = np.zeros(n_lipo_px, dtype = int)

ix = np.zeros(n_lipo_px, dtype = bool)
x0, x1 = fig.axes[0].get_xlim()
y0, y1 = fig.axes[0].get_ylim()
x = np.linspace(x0, x1, mask.shape[1])
y = np.linspace(y0, y1, mask.shape[0])

for i in range(n_lipo_px):
    if np.mod(i, 10) == 0:
        print(i)
    px = all_lipo_pixels[i]
    xi[i] = bisect_left(x, im_bin_npy2r_lipo[px[0], px[1], px[2]])
    yi[i] = bisect_left(y, im_bin_car4_lipo[px[0], px[1], px[2]])
    if mask[yi[i], xi[i]] > 0:
        ix[i] = True

all_lipo_pixels = all_lipo_pixels[ix]

x_vals = [im_bin_npy2r_lipo[a[0], a[1], a[2]] for a in all_lipo_pixels]
y_vals = [im_bin_car4_lipo[a[0], a[1], a[2]] for a in all_lipo_pixels]

plt.scatter(x_vals, y_vals, color = 'r')

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310


## Find mahalanobis distance of pixels from lipofuscin cloud

In [34]:
#img_dict = {'npy2r': im_bin_npy2r_lipo, 'car4': im_bin_car4_lipo}
img_dict = {'npy2r': im_bin_npy2r_lipo, 'tac2': im_bin_tac2_lipo, 'car4': im_bin_car4_lipo}
#img_dict = {'npy2r': im_diff_npy2r, 'tac2': im_diff_tac2, 'car4': im_diff_car4}
#img_dict = {'npy2r': im_array_npy2r, 'tac2': im_array_tac2, 'car4': im_array_car4}

pixels_dict = {'Cells': all_cell_pixels, 'Lipofuscin': all_lipo_pixels}

origin_group = 'Lipofuscin'
bin_group = 'Cells'

colors_dict = {'Cells': 'b', 'Lipofuscin': 'k'}

title = 'Diff gauss sigma = ({0}, {1}); binarized above {2} std from median'.format(sigma_small, sigma_large, thresh_scale)
#title = 'Raw images'
save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR 3.11\\S3\\Plots'
save_file = '{0} hist lipo + cells.png'.format(title)
#save_file = '{0} hist cells.png'.format(title)

output = m_dist.m_dist(img_dict, pixels_dict, origin_group, bin_group, colors_dict = colors_dict, thresh_scale = 1.5,
                            title = title, save = True, save_loc = save_loc, save_file = save_file)

m_dist_vals = output['m_dist']
thresh = output['thresh']
cov = output['cov']

print('Threshold log mahalanobis distance = {0}'.format(np.log(thresh)))

del img_dict
del pixels_dict

Threshold log mahalanobis distance = 1.4897658655476327


##### Not necessary to run this cell - it's a part of m_dist.py


In [37]:

channel_names = list(img_dict.keys())
n_channels = len(channel_names)

px_groups = list(pixels_dict.keys())


# %% Get values for each channel for each pixel group
px_vals = {}

for group in px_groups:

    px = pixels_dict[group]


    px_vals[group] = {}

    for channel in channel_names:
        px_vals[group][channel] = img_dict[channel][(px[:, 0], px[:, 1], px[:, 2])]


# %% Get mean and covariance matrix of origin group
mu = np.zeros(n_channels)
origin_px = np.zeros([n_channels, px_vals[origin_group][channel_names[0]].shape[0]])
for c in range(n_channels):
    mu[c] = np.mean(px_vals[origin_group][channel_names[c]])
    origin_px[c, :] = px_vals[origin_group][channel_names[c]]

cov = np.cov(origin_px) 
VI = np.linalg.inv(cov)     

In [41]:
plt.figure()
plt.imshow(np.log(cov))
plt.colorbar(label = 'Log covariance')

## Use mahalanobis distance to label all lipofuscin pixels 

In [35]:
lipo_pixels = all_cell_pixels[m_dist_vals['Cells'] < thresh, :]

## Don't exclude pixels below some average intensity

In [36]:
lipo_vals = np.zeros([3, len(lipo_pixels)])
lipo_vals[0, :] = np.array([im_bin_npy2r_lipo[a[0], a[1], a[2]] for a in lipo_pixels])
lipo_vals[1, :] = np.array([im_bin_car4_lipo[a[0], a[1], a[2]] for a in lipo_pixels])
lipo_vals[2, :] = np.array([im_bin_tac2_lipo[a[0], a[1], a[2]] for a in lipo_pixels])

mean_vals = np.mean(lipo_vals, axis = 0)
print(mean_vals.shape)

lipo_low_cutoff = np.exp(5)
lipo_pixels = lipo_pixels[mean_vals > lipo_low_cutoff] 

im_array_lipo = np.zeros(im_array_npy2r.shape)
im_array_lipo[lipo_pixels[:, 0], lipo_pixels[:, 1], lipo_pixels[:, 2]] = np.ones(lipo_pixels.shape[0])

(615783,)


## Re-calculate cell masks excluding lipofuscin pixels

In [55]:
save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR 3.12\S3'
save_file = 'HCR_3.12_S3_cell_masks_without_lipofuscin.pkl'

try:
    with open('{0}{1}{2}'.format(save_loc, sep, save_file), 'rb') as f:
        cell_pixels_no_lipo = pkl.load(f)
        print('Data loaded')
        jhk
except:
    
    print('Calculating')
    cell_pixels_no_lipo = {}

#     cells = mask_vertices.keys()
    cells = Cell.cell_data.keys()
#     cells = cells_sub + 1
    n_px_total = 0
    for cell in cells:

        data = Cell.cell_data[cell]
        planes = data['z_planes']
        cell_pixels_no_lipo[cell] = {}

        for plane in planes:

            x = mask_vertices[cell][plane][0]
            y = mask_vertices[cell][plane][1]
            n = len(x)
            cell_pixels_no_lipo[cell][plane] = np.array([[x[i], y[i]] for i in range(n) if im_array_lipo[plane, x[i], y[i]] == 0])
            cell_pixels_no_lipo[cell][plane] = np.transpose(cell_pixels_no_lipo[cell][plane])
#             cell_pixels_no_lipo[cell][plane] = {}
#             n_px = mask_vertices[cell][plane][0].shape[0]
#             m = m_dist_vals['Cells'][n_px_total:n_px_total + n_px] # Mahalanobis distance values for pixels in cell mask
#             px_coords = np.array(mask_vertices[cell][plane])
#             cell_pixels_no_lipo[cell][plane] = px_coords[:, m >= thresh]
#             #cell_pixels_no_lipo[cell][plane] = px_coords[:, m ]
#             n_px_total += n_px
            


    if not n_px_total == all_cell_pixels.shape[0]:
        print('Total number of pixels incorrect, could have removed incorrect pixels for cell')\

    with open('{0}{1}{2}'.format(save_loc, sep, save_file), 'wb') as f:
        pkl.dump(cell_pixels_no_lipo, f)

Data loaded
Calculating
Total number of pixels incorrect, could have removed incorrect pixels for cell


In [39]:
plane

0.0

## Delete large arrays

In [ ]:
del im_diff_npy2r
del im_diff_tac2
del im_diff_car4
del im_bin_npy2r
del im_bin_car4
del im_bin_tac2


In [ ]:
del im_bin_vip
del im_diff_vip

In [ ]:
del im_array_npy2r
del im_array_tac2
del im_array_car4
del im_array_vip
del im_array_dapi

# Classify cells by gene expression

## Calculate intensity for each gene in each cell 

### Average intensity in all pixels in cell mask

In [52]:
save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR 3.12\\S3'
save_file = 'S3_data_npy2r_tac2_car4.pkl'

try:
    print('Warning: recalculate if including all cells')
    with open('{0}\\{1}'.format(save_loc, save_file), 'rb') as f:
        sig = pkl.load(f)
    avg_intensity_npy2r = sig['npy2r']
    avg_intensity_tac2 = sig['tac2'] 
    avg_intensity_car4 = sig['car4']
    print('Data loaded')
    fj


except:
    print('Data not found, calculating intensities')
    no_cells = len(Cell.cell_data.keys())
    avg_intensity_npy2r = np.zeros(no_cells)
    avg_intensity_car4 = np.zeros(no_cells)
    avg_intensity_tac2 = np.zeros(no_cells)

#     for cell in range(no_cells):
    for cell in np.array(list(Cell.cell_data.keys())) - 1:
#     for cell in cells_sub:
        
        if np.mod(cell, 10) == 0:
            print('Cell {0}'.format(cell))
        cell_no = cell + 1
        cell_dict = Cell.cell_data[cell_no]
        z_planes = cell_dict['z_planes']

        intensity_npy2r = np.zeros(cell_dict['no_planes'])
        intensity_car4 = np.zeros(cell_dict['no_planes'])
        intensity_tac2 = np.zeros(cell_dict['no_planes'])

        i = 0
        for plane in z_planes:

            xy_coord = mask_vertices[cell_no][plane]
            x = xy_coord[0]
            y = xy_coord[1]
            intensity_npy2r[i] = np.mean(im_array_npy2r[plane, x, y])
            intensity_car4[i] = np.mean(im_array_car4[plane, x, y])
            intensity_tac2[i] = np.mean(im_array_tac2[plane, x, y])
            i += 1

        avg_intensity_npy2r[cell] = np.mean(intensity_npy2r)
        avg_intensity_car4[cell] = np.mean(intensity_car4)
        avg_intensity_tac2[cell] = np.mean(intensity_tac2)

        with open('{0}\\{1}'.format(save_loc, save_file), 'wb') as f:
            pkl.dump({'npy2r': avg_intensity_npy2r, 'tac2': avg_intensity_tac2, 'car4':avg_intensity_car4}, f)


Data loaded
Data not found, calculating intensities
Cell 0
Cell 10
Cell 20
Cell 30
Cell 40
Cell 50
Cell 60
Cell 70
Cell 80
Cell 90


### Intensity in binarized images in all cell pixels

In [53]:
save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR 3.12\\S3'
save_file = 'S3_data_npy2r_tac2_car4.pkl'

try:
    with open('{0}\\{1}'.format(save_loc, save_file), 'rb') as f:
        sig = pkl.load(f)
    int_npy2r_bin_all = sig['npy2r_bin_all']
    int_tac2_bin_all = sig['tac2_bin_all'] 
    int_car4_bin_all = sig['car4_bin_all']
    print('Data loaded')
    jio

except:
    print('Data not found, calculating intensities')
    no_cells = len(Cell.cell_data.keys())
    int_npy2r_bin_all = np.zeros(no_cells)
    int_tac2_bin_all = np.zeros(no_cells)
    int_car4_bin_all = np.zeros(no_cells)

    for cell in range(no_cells):
#     for cell in cells_sub:
        
        if np.mod(cell, 10) == 0:
            print('Cell {0}'.format(cell))
        cell_no = cell + 1
        cell_dict = Cell.cell_data[cell_no]
        z_planes = cell_dict['z_planes']

        intensity_npy2r = np.zeros(cell_dict['no_planes'])
        intensity_car4 = np.zeros(cell_dict['no_planes'])
        intensity_tac2 = np.zeros(cell_dict['no_planes'])

        i = 0
        for plane in z_planes:

            xy_coord = mask_vertices[cell_no][plane]
            x = xy_coord[0]
            y = xy_coord[1]
            intensity_npy2r[i] = np.mean(im_bin_npy2r[plane, x, y])
            intensity_car4[i] = np.mean(im_bin_car4[plane, x, y])
            intensity_tac2[i] = np.mean(im_bin_tac2[plane, x, y])
            i += 1

        int_npy2r_bin_all[cell] = np.mean(intensity_npy2r)
        int_car4_bin_all[cell] = np.mean(intensity_car4)
        int_tac2_bin_all[cell] = np.mean(intensity_tac2)

        with open('{0}\\{1}'.format(save_loc, save_file), 'wb') as f:
            pkl.dump({'npy2r_bin_all': int_npy2r_bin_all, 
                      'tac2_bin_all': int_tac2_bin_all, 
                      'car4_bin_all':int_car4_bin_all}, f)


Data not found, calculating intensities
Cell 0


NameError: name 'im_bin_npy2r' is not defined

### Intensity in masks excluding lipofuscin pixels

In [56]:
save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR 3.12\\S3'
save_file = 'S3_data_npy2r_tac2_car4.pkl'

try:
    print('Warning: recalculate if including all cells')

    with open('{0}\\{1}'.format(save_loc, save_file), 'rb') as f:
        sig = pkl.load(f)
    int_npy2r = sig['npy2r_lipo_excl']
    int_tac2 = sig['tac2_lipo_excl'] 
    int_car4 = sig['car4_lipo_excl']
    print('Data loaded')
    fsl

except:
    print('Data not found, calculating intensities')
    no_cells = len(Cell.cell_data.keys())
    int_npy2r = np.zeros(no_cells)
    int_car4 = np.zeros(no_cells)
    int_tac2 = np.zeros(no_cells)

    for cell in range(no_cells):
#     for cell in cells_sub:
        
        if np.mod(cell, 10) == 0:
            print('Cell {0}'.format(cell))
        cell_no = cell + 1
        cell_dict = Cell.cell_data[cell_no]
        z_planes = cell_dict['z_planes']

        intensity_npy2r = np.zeros(cell_dict['no_planes'])
        intensity_car4 = np.zeros(cell_dict['no_planes'])
        intensity_tac2 = np.zeros(cell_dict['no_planes'])

        i = 0
        for plane in z_planes:

            xy_coord = cell_pixels_no_lipo[cell_no][plane]
            if xy_coord.shape[1] == 0: # all pixels are lipofuscin pixels
                intensity_npy2r[i] = 0
                intensity_car4[i] = 0
                intensity_tac2[i] = 0
                continue
                
            x = xy_coord[0, :]
            y = xy_coord[1, :]
            intensity_npy2r[i] = np.mean(im_array_npy2r[plane, x, y])
            intensity_car4[i] = np.mean(im_array_car4[plane, x, y])
            intensity_tac2[i] = np.mean(im_array_tac2[plane, x, y])
            i += 1

        int_npy2r[cell] = np.mean(intensity_npy2r)
        int_car4[cell] = np.mean(intensity_car4)
        int_tac2[cell] = np.mean(intensity_tac2)
        
        sig['npy2r_lipo_excl'] = int_npy2r
        sig['tac2_lipo_excl'] = int_tac2 
        sig['car4_lipo_excl'] = int_car4 

        with open('{0}\\{1}'.format(save_loc, save_file), 'wb') as f:
            pkl.dump(sig, f)


Data loaded
Data not found, calculating intensities
Cell 0
Cell 10
Cell 20
Cell 30
Cell 40
Cell 50
Cell 60
Cell 70
Cell 80
Cell 90


### Intensity in masks excluding lipofuscin pixels in binarized images (essentially spot counting)

#### Filter images

In [57]:
t0 = time.time()    

print('No images found, filtering images (~5 minutes)')
sigma_small = 1
sigma_large = 2


img = im_array_npy2r
im_diff_npy2r = diff_gauss.diff_gauss(sigma_small, sigma_large, img, do_plot = 1)
t1 = time.time() - t0
print('{0} seconds'.format(int(t1)))

img = im_array_tac2
im_diff_tac2 = diff_gauss.diff_gauss(sigma_small, sigma_large, img, do_plot = 0)
t1 = time.time() - t0
print('{0} seconds'.format(int(t1)))

img = im_array_car4
im_diff_car4 = diff_gauss.diff_gauss(sigma_small, sigma_large, img, do_plot = 0)
t1 = time.time() - t0
print('{0} seconds'.format(int(t1)))

#     img = im_array_vip
#     im_diff_vip = diff_gauss.diff_gauss(sigma_small, sigma_large, img, do_plot = 1)
#     t1 = time.time() - t0
#     print('{0} seconds'.format(int(t1)))

del img

No images found, filtering images (~5 minutes)
98 seconds
197 seconds
296 seconds


#### Binarize

In [58]:
img_dict = {'npy2r': im_diff_npy2r, 'tac2': im_diff_tac2, 'car4': im_diff_car4}
#img_dict = {'npy2r': im_array_npy2r, 'tac2': im_array_tac2, 'car4': im_array_car4}

pixels_dict = {'Cells': all_cell_pixels, 'Lipofuscin': all_lipo_pixels}
#pixels_dict = {'Cells': all_cell_pixels}

colors_dict = {'Cells': 'b', 'Lipofuscin': 'k'}

title = 'Diff gauss sigma = ({0}, {1})'.format(sigma_small, sigma_large)
#title = 'Raw images'
save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR 3.11\\S3\\Plots'
save_file = '{0} hist lipo + cells.png'.format(title)
#save_file = '{0} hist cells.png'.format(title)

thresh_scale = 12

thresh = hist_channels.hist_channels(img_dict, pixels_dict, max_per_group= 100000, 
                                     do_bin = True, bin_group = 'Cells', thresh_scale = thresh_scale,
                                     colors_dict = colors_dict, 
                                     title = title, save = True, save_loc = save_loc, save_file = save_file)

del img_dict
del pixels_dict

im_bin_npy2r = np.zeros(im_diff_npy2r.shape)
im_bin_tac2 = np.zeros(im_diff_npy2r.shape)
im_bin_car4 = np.zeros(im_diff_npy2r.shape)
#im_bin_vip = np.zeros(im_diff_vip.shape)

ix_npy2r = np.where(im_diff_npy2r > thresh['npy2r'])
ix_tac2 = np.where(im_diff_tac2 > thresh['tac2'])
ix_car4 = np.where(im_diff_car4 > thresh['car4'])
#ix_vip = np.where(im_diff_vip > thresh['vip'])

im_bin_npy2r[ix_npy2r] = im_diff_npy2r[ix_npy2r]
im_bin_tac2[ix_tac2] = im_diff_tac2[ix_tac2]
im_bin_car4[ix_car4] = im_diff_car4[ix_car4]
#im_bin_vip[ix_vip] = im_diff_vip[ix_vip]

Finding threshold for binarizing
Finding threshold for binarizing
Finding threshold for binarizing


#### Calculate intensities

In [59]:
# Note: size of spot is also relevant here. 
save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR 3.12\\S3'
save_file = 'S3_data_npy2r_tac2_car4.pkl'

try:
    with open('{0}\\{1}'.format(save_loc, save_file), 'rb') as f:
        sig = pkl.load(f)
    int_npy2r_bin = sig['npy2r_lipo_excl_bin']
    int_tac2_bin = sig['tac2_lipo_excl_bin'] 
    int_car4_bin = sig['car4_lipo_excl_bin']
    print('Data loaded')
    jfg

except:
    print('Data not found, calculating intensities')
    no_cells = len(Cell.cell_data.keys())
    int_npy2r_bin = np.zeros(no_cells)
    int_car4_bin = np.zeros(no_cells)
    int_tac2_bin = np.zeros(no_cells)

    for cell in range(no_cells):
#     for cell in cells_sub:
        
        if np.mod(cell, 10) == 0:
            print('Cell {0}'.format(cell))
        cell_no = cell + 1
        cell_dict = Cell.cell_data[cell_no]
        z_planes = cell_dict['z_planes']

        intensity_npy2r = np.zeros(cell_dict['no_planes'])
        intensity_car4 = np.zeros(cell_dict['no_planes'])
        intensity_tac2 = np.zeros(cell_dict['no_planes'])

        i = 0
        for plane in z_planes:

            xy_coord = cell_pixels_no_lipo[cell_no][plane]
            
            if xy_coord.shape[1] == 0: # all pixels are lipofuscin pixels
                intensity_npy2r[i] = 0
                intensity_car4[i] = 0
                intensity_tac2[i] = 0
                continue
                
            x = xy_coord[0, :]
            y = xy_coord[1, :]
            intensity_npy2r[i] = np.mean(im_bin_npy2r[plane, x, y])
            intensity_car4[i] = np.mean(im_bin_car4[plane, x, y])
            intensity_tac2[i] = np.mean(im_bin_tac2[plane, x, y])
            i += 1

        int_npy2r_bin[cell] = np.mean(intensity_npy2r)
        int_car4_bin[cell] = np.mean(intensity_car4)
        int_tac2_bin[cell] = np.mean(intensity_tac2)
        
    sig['npy2r_lipo_excl_bin'] = int_npy2r_bin
    sig['tac2_lipo_excl_bin'] = int_tac2_bin 
    sig['car4_lipo_excl_bin'] = int_car4_bin 

    with open('{0}\\{1}'.format(save_loc, save_file), 'wb') as f:
        pkl.dump(sig, f)



Data not found, calculating intensities
Cell 0
Cell 10
Cell 20
Cell 30
Cell 40
Cell 50
Cell 60
Cell 70
Cell 80
Cell 90


##### Calculate intensities - old

In [119]:
# Note: size of spot is also relevant here. 
save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR 3.12\\S3'
save_file = 'S3_data_npy2r_tac2_car4.pkl'

try:
    with open('{0}\\{1}'.format(save_loc, save_file), 'rb') as f:
        sig = pkl.load(f)
    int_npy2r_bin = sig['npy2r_lipo_excl_bin']
    int_tac2_bin = sig['tac2_lipo_excl_bin'] 
    int_car4_bin = sig['car4_lipo_excl_bin']
    kgl
    print('Data loaded')

except:
    print('Data not found, calculating intensities')
    no_cells = len(Cell.cell_data.keys())
    int_npy2r_bin = np.zeros(no_cells)
    int_car4_bin = np.zeros(no_cells)
    int_tac2_bin = np.zeros(no_cells)

    for cell in range(no_cells):

        if np.mod(cell, 10) == 0:
            print('Cell {0}'.format(cell))
        cell_no = cell + 1
        cell_dict = Cell.cell_data[cell_no]
        z_planes = cell_dict['z_planes']

        intensity_npy2r = np.zeros(cell_dict['no_planes'])
        intensity_car4 = np.zeros(cell_dict['no_planes'])
        intensity_tac2 = np.zeros(cell_dict['no_planes'])

        i = 0
        for plane in z_planes:

            xy_coord = cell_pixels_no_lipo[cell_no][plane]
            x = xy_coord[0, :]
            y = xy_coord[1, :]
            intensity_npy2r[i] = np.mean(im_bin_npy2r[plane, x, y])
            intensity_car4[i] = np.mean(im_bin_car4[plane, x, y])
            intensity_tac2[i] = np.mean(im_bin_tac2[plane, x, y])
            i += 1

        int_npy2r_bin[cell] = np.mean(intensity_npy2r)
        int_car4_bin[cell] = np.mean(intensity_car4)
        int_tac2_bin[cell] = np.mean(intensity_tac2)
        
    sig['npy2r_lipo_excl_bin'] = int_npy2r_bin
    sig['tac2_lipo_excl_bin'] = int_tac2_bin 
    sig['car4_lipo_excl_bin'] = int_car4_bin 

    with open('{0}\\{1}'.format(save_loc, save_file), 'wb') as f:
        pkl.dump(sig, f)



Data not found, calculating intensities
Cell 0
Cell 10
Cell 20
Cell 30
Cell 40
Cell 50
Cell 60
Cell 70
Cell 80
Cell 90
Cell 100
Cell 110
Cell 120
Cell 130
Cell 140
Cell 150
Cell 160
Cell 170
Cell 180
Cell 190
Cell 200
Cell 210
Cell 220
Cell 230
Cell 240
Cell 250
Cell 260
Cell 270
Cell 280
Cell 290
Cell 300
Cell 310
Cell 320


### Intensity in masks excluding lipofuscin pixels in diff images

In [69]:
# Note: size of spot is also relevant here. 

save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR 3.12\\S3'
save_file = 'S3_data2_npy2r_tac2_car4.pkl'

try:
    with open('{0}\\{1}'.format(save_loc, save_file), 'rb') as f:
        sig = pkl.load(f)
    int_npy2r_diff = sig['npy2r_lipo_excl_diff']
    int_tac2_diff = sig['tac2_lipo_excl_diff'] 
    int_car4_diff = sig['car4_lipo_excl_diff']
    print('Data loaded')

except:
    print('Data not found, calculating intensities')
    no_cells = len(Cell.cell_data.keys())
    int_npy2r_diff = np.zeros(no_cells)
    int_car4_diff = np.zeros(no_cells)
    int_tac2_diff = np.zeros(no_cells)

    for cell in range(no_cells):

        if np.mod(cell, 10) == 0:
            print('Cell {0}'.format(cell))
        cell_no = cell + 1
        cell_dict = Cell.cell_data[cell_no]
        z_planes = cell_dict['z_planes']

        intensity_npy2r = np.zeros(cell_dict['no_planes'])
        intensity_car4 = np.zeros(cell_dict['no_planes'])
        intensity_tac2 = np.zeros(cell_dict['no_planes'])

        i = 0
        for plane in np.array(z_planes).astype(int):

            xy_coord = cell_pixels_no_lipo[cell_no][plane]
            x = xy_coord[0]
            y = xy_coord[1]
            intensity_npy2r[i] = np.mean(im_diff_npy2r[plane, x, y])
            intensity_car4[i] = np.mean(im_diff_car4[plane, x, y])
            intensity_tac2[i] = np.mean(im_diff_tac2[plane, x, y])
            i += 1

        int_npy2r_diff[cell] = np.mean(intensity_npy2r)
        int_car4_diff[cell] = np.mean(intensity_car4)
        int_tac2_diff[cell] = np.mean(intensity_tac2)
        
        sig['npy2r_lipo_excl_bin'] = int_npy2r_diff
        sig['tac2_lipo_excl_bin'] = int_tac2_diff
        sig['car4_lipo_excl_bin'] = int_car4_diff

    with open('{0}\\{1}'.format(save_loc, save_file), 'wb') as f:
            pkl.dump(sig, f)


Data not found, calculating intensities
Cell 0
Cell 10
Cell 20
Cell 30
Cell 40
Cell 50
Cell 60
Cell 70
Cell 80
Cell 90
Cell 100
Cell 110
Cell 120
Cell 130
Cell 140
Cell 150
Cell 160
Cell 170
Cell 180
Cell 190
Cell 200
Cell 210
Cell 220
Cell 230
Cell 240
Cell 250
Cell 260
Cell 270
Cell 280
Cell 290
Cell 300
Cell 310
Cell 320


## Local background

### Raw images

In [62]:
n = len(plane_nos)

In [63]:
save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR 3.12\\S3'
save_file = 'S3_background_npy2r_tac2_car4.pkl'

try:
    with open('{0}\\{1}'.format(save_loc, save_file), 'rb') as f:
        dict = pkl.load(f)
    
    avg_bg_npy2r = dict['npy2r'] 
    avg_bg_tac2 = dict['tac2']  
    avg_bg_car4 = dict['car4']
    centers = dict['Centers']
    n_vert_all = dict['n_vert_all']
    #bg_all = dict['bg_all']
    jkl
    print('Data loaded')

except:
    
    print('Data not found, calculating background values')
    bg_all = np.ones([n, h, w])
    n_vert_all = {}
    centers = {}

    no_cells = len(Cell.cell_data.keys())
    for cell in range(no_cells):

        cell_no = cell + 1
        cell_dict = Cell.cell_data[cell_no]
        z_planes = cell_dict['z_planes']
        n_vert_all[cell] = {}
        centers[cell] = {}

        for plane in z_planes:

            xy_coord = mask_vertices[cell_no][plane]
            x = xy_coord[0]
            y = xy_coord[1]
            z = plane
            centers[cell][plane] = np.array([np.mean(x), np.mean(y)]).astype(int)


            bg_all[z, x, y] = np.zeros([1, len(x)])

            n_vert_all[cell][plane] = len(x) 
            
    # fig, ax = plt.subplots(nrows = 3, ncols = 20)
    um_per_px = 0.13
    no_cells = len(Cell.cell_data.keys())
    avg_bg_npy2r = np.zeros(no_cells)
    avg_bg_car4 = np.zeros(no_cells)
    avg_bg_tac2 = np.zeros(no_cells)

    local_region = 50 # in um
    local_px = int(local_region/um_per_px/2)

    min_dist = 10 # in um
    min_dist_px = min_dist/um_per_px

    for cell in range(no_cells):
        cell_no = cell + 1
        cell_dict = Cell.cell_data[cell_no]
        z_planes = cell_dict['z_planes']

        bg_npy2r = np.zeros(len(z_planes))
        bg_car4 = np.zeros(len(z_planes))
        bg_tac2 = np.zeros(len(z_planes))

        p = 0
        for plane in z_planes:

            center = centers[cell][plane]
            n_verts = n_vert_all[cell][plane]

            x1 = np.max([0, center[0] - local_px])
            x2 = np.min([h, center[0] + local_px])
            y1 = np.max([0, center[1] - local_px])
            y2 = np.min([w, center[1] + local_px])

            center_local = [local_px, local_px]

            local_bg = bg_all[plane, x1:x2, y1:y2].astype(bool)
            local_im_npy2r = im_array_npy2r[plane, x1:x2, y1:y2]
            local_im_tac2 = im_array_tac2[plane, x1:x2, y1:y2]
            local_im_car4 = im_array_car4[plane, x1:x2, y1:y2]

    #         ax[0, cell].imshow(local_im, vmin = np.min(local_im), vmax = np.max(local_im)/2)

    #         ax[1, cell].imshow(local_bg)

            verts = np.array(np.where(local_bg))
            dist_from_center = np.linalg.norm(verts - np.reshape(center_local, [2, 1]), axis = 0)
            verts = verts[:, dist_from_center > min_dist_px]
            dist_from_center = dist_from_center[dist_from_center > min_dist_px]

            order = np.argsort(dist_from_center)
            n_verts = np.min([n_verts, len(order)])

            x_bg = verts[0][order[0:n_verts]]
            y_bg = verts[1][order[0:n_verts]]

            local_bg = np.zeros(local_bg.shape).astype(bool)
            local_bg[x_bg, y_bg] = np.ones(n_verts)

    #         ax[2, cell].imshow(local_bg)

            bg_npy2r[p] = np.mean(local_im_npy2r[local_bg])
            bg_car4[p] = np.mean(local_im_car4[local_bg])
            bg_tac2[p] = np.mean(local_im_tac2[local_bg])

            p += 1



        avg_bg_npy2r[cell] = np.mean(bg_npy2r)
        avg_bg_car4[cell] = np.mean(bg_car4)
        avg_bg_tac2[cell] = np.mean(bg_tac2)

    with open('{0}\\{1}'.format(save_loc, save_file), 'wb') as f:
        pkl.dump({'npy2r': avg_bg_npy2r, 'tac2': avg_bg_tac2, 'car4':avg_bg_car4, 'Centers': centers, 
                  'n_vert_all':n_vert_all, 
                  #'bg_all': bg_all
                 }, f)
            
    #del bg_all

Data not found, calculating background values


### Binarized images

In [28]:
save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR 3.11\\S3'
save_file = 'S3_background_npy2r_tac2_car4.pkl'

with open('{0}\\{1}'.format(save_loc, save_file), 'rb') as f:
        bg_dict = pkl.load(f)  

In [64]:
save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR 3.12\\S3'
save_file = 'S3_background_npy2r_tac2_car4.pkl'

try:
    with open('{0}\\{1}'.format(save_loc, save_file), 'rb') as f:
        bg_dict = pkl.load(f)   
    
    centers = bg_dict['Centers']
    n_vert_all = bg_dict['n_vert_all']
    #bg_all = bg_dict['bg_all']
    avg_bg_npy2r_bin = bg_dict['npy2r_bin'] 
    avg_bg_tac2_bin = bg_dict['tac2_bin']  
    avg_bg_car4_bin = bg_dict['car4_bin']
    hkfl

    print('Data loaded')

except:
    
    print('Data not found, calculating background values')
    bg_all = np.ones([n, h, w])
    n_vert_all = {}
    centers = {}

    no_cells = len(Cell.cell_data.keys())
    for cell in range(no_cells):
#     for cell in cells_sub:
        
        cell_no = cell + 1
        cell_dict = Cell.cell_data[cell_no]
        z_planes = cell_dict['z_planes']
        n_vert_all[cell] = {}
        centers[cell] = {}

        for plane in z_planes:

            xy_coord = mask_vertices[cell_no][plane]
            x = xy_coord[0]
            y = xy_coord[1]
            z = plane
            centers[cell][plane] = np.array([np.mean(x), np.mean(y)]).astype(int)


            bg_all[z, x, y] = np.zeros([1, len(x)])

            n_vert_all[cell][plane] = len(x) 
            
    # fig, ax = plt.subplots(nrows = 3, ncols = 20)
    um_per_px = 0.13
    no_cells = len(Cell.cell_data.keys())
    avg_bg_npy2r_bin = np.zeros(no_cells)
    avg_bg_car4_bin = np.zeros(no_cells)
    avg_bg_tac2_bin = np.zeros(no_cells)

    local_region = 50 # in um
    local_px = int(local_region/um_per_px/2)

    min_dist = 10 # in um
    min_dist_px = min_dist/um_per_px

    for cell in range(no_cells):
#     for cell in cells_sub:
        
        cell_no = cell + 1
        if np.mod(cell, 10) == 0:
            print('Cell {0}'.format(cell))
        cell_dict = Cell.cell_data[cell_no]
        z_planes = cell_dict['z_planes']

        bg_npy2r = np.zeros(len(z_planes))
        bg_car4 = np.zeros(len(z_planes))
        bg_tac2 = np.zeros(len(z_planes))

        p = 0
        for plane in z_planes:

            center = centers[cell][plane]
            n_verts = n_vert_all[cell][plane]

            x1 = np.max([0, center[0] - local_px])
            x2 = np.min([h, center[0] + local_px])
            y1 = np.max([0, center[1] - local_px])
            y2 = np.min([w, center[1] + local_px])

            center_local = [local_px, local_px]

            local_bg = bg_all[plane, x1:x2, y1:y2].astype(bool)
            local_im_npy2r = im_bin_npy2r[plane, x1:x2, y1:y2]
            local_im_tac2 = im_bin_tac2[plane, x1:x2, y1:y2]
            local_im_car4 = im_bin_car4[plane, x1:x2, y1:y2]

    #         ax[0, cell].imshow(local_im, vmin = np.min(local_im), vmax = np.max(local_im)/2)

    #         ax[1, cell].imshow(local_bg)

            verts = np.array(np.where(local_bg))
            dist_from_center = np.linalg.norm(verts - np.reshape(center_local, [2, 1]), axis = 0)
            verts = verts[:, dist_from_center > min_dist_px]
            dist_from_center = dist_from_center[dist_from_center > min_dist_px]

            order = np.argsort(dist_from_center)
            n_verts = np.min([n_verts, len(order)])

            x_bg = verts[0][order[0:n_verts]]
            y_bg = verts[1][order[0:n_verts]]

            local_bg = np.zeros(local_bg.shape).astype(bool)
            local_bg[x_bg, y_bg] = np.ones(n_verts)

    #         ax[2, cell].imshow(local_bg)

            bg_npy2r[p] = np.mean(local_im_npy2r[local_bg])
            bg_car4[p] = np.mean(local_im_car4[local_bg])
            bg_tac2[p] = np.mean(local_im_tac2[local_bg])

            p += 1



        avg_bg_npy2r_bin[cell] = np.mean(bg_npy2r)
        avg_bg_car4_bin[cell] = np.mean(bg_car4)
        avg_bg_tac2_bin[cell] = np.mean(bg_tac2)
        
    bg_dict['npy2r_bin'] = avg_bg_npy2r_bin 
    bg_dict['tac2_bin'] = avg_bg_tac2_bin 
    bg_dict['car4_bin'] = avg_bg_car4_bin 

    with open('{0}\\{1}'.format(save_loc, save_file), 'wb') as f:
        pkl.dump(bg_dict, f)
            
    #del bg_all

Data not found, calculating background values
Cell 0
Cell 10
Cell 20
Cell 30
Cell 40
Cell 50
Cell 60
Cell 70
Cell 80
Cell 90


### Diff images

In [71]:
save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR 3.12\\S3'
save_file = 'S3_background2_npy2r_tac2_car4.pkl'

try:
    with open('{0}\\{1}'.format(save_loc, save_file), 'rb') as f:
        bg_dict = pkl.load(f)   
    
    centers = bg_dict['Centers']
    n_vert_all = bg_dict['n_vert_all']
    bg_all = bg_dict['bg_all']
    avg_bg_npy2r_diff = bg_dict['npy2r_diff'] 
    avg_bg_tac2_diff = bg_dict['tac2_diff']  
    avg_bg_car4_diff = bg_dict['car4_diff']

    print('Data loaded')

except:
    
    print('Data not found, calculating background values')
    bg_all = np.ones([n, h, w])
    n_vert_all = {}
    centers = {}

    no_cells = len(Cell.cell_data.keys())
    for cell in range(no_cells):

        cell_no = cell + 1
        cell_dict = Cell.cell_data[cell_no]
        z_planes = cell_dict['z_planes']
        n_vert_all[cell] = {}
        centers[cell] = {}

        for plane in np.array(z_planes).astype(int):

            xy_coord = mask_vertices[cell_no][plane]
            x = xy_coord[0]
            y = xy_coord[1]
            z = plane
            centers[cell][plane] = np.array([np.mean(x), np.mean(y)]).astype(int)


            bg_all[z, x, y] = np.zeros([1, len(x)])

            n_vert_all[cell][plane] = len(x) 
            
    # fig, ax = plt.subplots(nrows = 3, ncols = 20)
    um_per_px = 0.19
    no_cells = len(Cell.cell_data.keys())
    avg_bg_npy2r_diff = np.zeros(no_cells)
    avg_bg_car4_diff = np.zeros(no_cells)
    avg_bg_tac2_diff = np.zeros(no_cells)

    local_region = 50 # in um
    local_px = int(local_region/um_per_px/2)

    min_dist = 10 # in um
    min_dist_px = min_dist/um_per_px

    for cell in range(no_cells):
        cell_no = cell + 1
        if np.mod(cell, 10) == 0:
            print('Cell {0}'.format(cell))
        cell_dict = Cell.cell_data[cell_no]
        z_planes = cell_dict['z_planes']

        bg_npy2r = np.zeros(len(z_planes))
        bg_car4 = np.zeros(len(z_planes))
        bg_tac2 = np.zeros(len(z_planes))

        p = 0
        for plane in np.array(z_planes).astype(int):

            center = centers[cell][plane]
            n_verts = n_vert_all[cell][plane]

            x1 = np.max([0, center[0] - local_px])
            x2 = np.min([h, center[0] + local_px])
            y1 = np.max([0, center[1] - local_px])
            y2 = np.min([w, center[1] + local_px])

            center_local = [local_px, local_px]

            local_bg = bg_all[plane, x1:x2, y1:y2].astype(bool)
            local_im_npy2r = im_diff_npy2r[plane, x1:x2, y1:y2]
            local_im_tac2 = im_diff_tac2[plane, x1:x2, y1:y2]
            local_im_car4 = im_diff_car4[plane, x1:x2, y1:y2]

    #         ax[0, cell].imshow(local_im, vmin = np.min(local_im), vmax = np.max(local_im)/2)

    #         ax[1, cell].imshow(local_bg)

            verts = np.array(np.where(local_bg))
            dist_from_center = np.linalg.norm(verts - np.reshape(center_local, [2, 1]), axis = 0)
            verts = verts[:, dist_from_center > min_dist_px]
            dist_from_center = dist_from_center[dist_from_center > min_dist_px]

            order = np.argsort(dist_from_center)
            n_verts = np.min([n_verts, len(order)])

            x_bg = verts[0][order[0:n_verts]]
            y_bg = verts[1][order[0:n_verts]]

            local_bg = np.zeros(local_bg.shape).astype(bool)
            local_bg[x_bg, y_bg] = np.ones(n_verts)

    #         ax[2, cell].imshow(local_bg)

            bg_npy2r[p] = np.mean(local_im_npy2r[local_bg])
            bg_car4[p] = np.mean(local_im_car4[local_bg])
            bg_tac2[p] = np.mean(local_im_tac2[local_bg])

            p += 1



        avg_bg_npy2r_diff[cell] = np.mean(bg_npy2r)
        avg_bg_car4_diff[cell] = np.mean(bg_car4)
        avg_bg_tac2_diff[cell] = np.mean(bg_tac2)
        
    bg_dict['npy2r_bin'] = avg_bg_npy2r_diff 
    bg_dict['tac2_bin'] = avg_bg_tac2_diff 
    bg_dict['car4_bin'] = avg_bg_car4_diff 

    with open('{0}\\{1}'.format(save_loc, save_file), 'wb') as f:
        pkl.dump(bg_dict, f)
            
    del bg_all

Data not found, calculating background values
Cell 0
Cell 10
Cell 20
Cell 30
Cell 40
Cell 50
Cell 60
Cell 70
Cell 80
Cell 90
Cell 100
Cell 110
Cell 120
Cell 130
Cell 140
Cell 150
Cell 160
Cell 170
Cell 180
Cell 190
Cell 200
Cell 210
Cell 220
Cell 230
Cell 240
Cell 250
Cell 260
Cell 270
Cell 280
Cell 290
Cell 300
Cell 310
Cell 320


### Calculate mask centers

In [83]:
bg_all = np.ones([n, h, w])
n_vert_all = {}
centers = {}

no_cells = len(Cell.cell_data.keys())
for cell in range(no_cells):

    cell_no = cell + 1
    cell_dict = Cell.cell_data[cell_no]
    z_planes = cell_dict['z_planes']
    n_vert_all[cell] = {}
    centers[cell] = {}

    for plane in z_planes:

        xy_coord = mask_vertices[cell_no][plane]
        x = xy_coord[0]
        y = xy_coord[1]
        z = plane
        centers[cell][plane] = np.array([np.mean(x), np.mean(y)]).astype(int)


        bg_all[z, x, y] = np.zeros([1, len(x)])

        n_vert_all[cell][plane] = len(x) 

In [78]:
save_file

'S1_background_ndnf_sst_vip.pkl'

In [94]:
with open('{0}\\{1}'.format(save_loc, save_file), 'wb') as f:
            pkl.dump({'npy2r': avg_bg_npy2r, 'tac2': avg_bg_tac2, 'car4':avg_bg_car4, 'Centers': centers,
                     'n_vert_all': n_vert_all, 'bg_all': bg_all}, f)

In [127]:
del all_cell_pixels

## Use background to estimate significance of signal

### Get bootstrap sample means from background

#### Raw images

In [65]:
t0 = time.time()

save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR 3.12\\S3'
save_file = 'S3_bootstrap.pkl'

# Parameters
n_sample = 1000 # Number of samples to calculate bootstrap confidence interval
um_per_px = 0.19
local_region = 50 # in um
min_dist = 10 # in um
bg_factor = 4 # Take 4x the number of pixels in the mask of the cell, for local background - it will still be local but you'll
            # get a better estimate of background by averaging over more pixels.

try:
    with open('{0}\\{1}'.format(save_loc, save_file), 'rb') as f:
        d = pkl.load(f)
    bg_bs_npy2r = d['npy2r']
    bg_bs_tac2 = d['tac2']
    bg_bs_car4 = d['car4']
    print('Data loaded')
    
except:
    print('No data found, calculating p values')

    no_cells = len(Cell.cell_data.keys())
    bg_bs_npy2r = {}
    bg_bs_tac2 = {}
    bg_bs_car4 = {}

    local_px = int(local_region/um_per_px/2)
    min_dist_px = min_dist/um_per_px

    for cell in range(no_cells):

        if np.mod(cell, 10) == 0:
            print('Cell {0}: {1} seconds'.format(cell, np.round(time.time() - t0)))

        cell_no = cell + 1
        cell_dict = Cell.cell_data[cell_no]
        z_planes = cell_dict['z_planes']

        bg_bs_npy2r[cell] = {}
        bg_bs_tac2[cell] = {}
        bg_bs_car4[cell] = {}
        
        p = 0

        for plane in z_planes:

            bg_bs_npy2r[cell][plane] = np.zeros(n_sample)
            bg_bs_tac2[cell][plane] = np.zeros(n_sample)
            bg_bs_car4[cell][plane] = np.zeros(n_sample)
            
            center = centers[cell][plane]
            n_verts = n_vert_all[cell][plane]

            x1 = np.max([0, center[0] - local_px])
            x2 = np.min([h, center[0] + local_px])
            y1 = np.max([0, center[1] - local_px])
            y2 = np.min([w, center[1] + local_px])

            center_local = [local_px, local_px]

            local_bg = bg_all[plane, x1:x2, y1:y2].astype(bool)

            verts = np.array(np.where(local_bg))
            dist_from_center = np.linalg.norm(verts - np.reshape(center_local, [2, 1]), 
                                              axis = 0)
            verts = verts[:, dist_from_center > min_dist_px]
            dist_from_center = dist_from_center[dist_from_center > min_dist_px]

            order = np.argsort(dist_from_center)
            n_verts_bg = np.min([bg_factor*n_verts, len(order)])

            verts = verts[:, order[0:n_verts_bg]]
            samples = np.random.choice(np.linspace(0, n_verts_bg - 1, n_verts_bg).astype(int), 
                                       (n_verts, n_sample), replace = True)
            x_bs = verts[0][samples]
            y_bs = verts[1][samples]

            local_im_npy2r = im_array_npy2r[plane, x1:x2, y1:y2]
            local_im_tac2 = im_array_tac2[plane, x1:x2, y1:y2]
            local_im_car4 = im_array_car4[plane, x1:x2, y1:y2]


            local_bg_bs = np.zeros([x2 - x1, y2 - y1]).astype(bool)
            for s in range(n_sample):
                local_bg_bs[x_bs[:, s], y_bs[:, s]] = np.ones(n_verts)
                bg_bs_npy2r[cell][plane][s] = np.mean(local_im_npy2r[local_bg_bs])
                bg_bs_car4[cell][plane][s] = np.mean(local_im_car4[local_bg_bs])
                bg_bs_tac2[cell][plane][s] = np.mean(local_im_tac2[local_bg_bs])

            p += 1

    with open('{0}\\{1}'.format(save_loc, save_file), 'wb') as f:
        pkl.dump({'npy2r': bg_bs_npy2r, 'car4': bg_bs_car4, 'tac2': bg_bs_tac2}, f)     
    

No data found, calculating p values
Cell 0: 0.0 seconds
Cell 10: 6.0 seconds
Cell 20: 14.0 seconds
Cell 30: 22.0 seconds
Cell 40: 29.0 seconds
Cell 50: 37.0 seconds
Cell 60: 44.0 seconds
Cell 70: 51.0 seconds
Cell 80: 57.0 seconds
Cell 90: 64.0 seconds


#### Binarized images

In [66]:
save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR 3.12\\S3'
save_file = 'S3_bootstrap.pkl'

# Parameters
n_sample = 1000 # Number of samples to calculate bootstrap confidence interval
um_per_px = 0.13
local_region = 50 # in um
min_dist = 10 # in um
bg_factor = 10 # Take 10x the number of pixels in the mask of the cell, for local background - it will still be local but you'll
            # get a better estimate of background by averaging over more pixels.

t0 = time.time()
try:
    with open('{0}\\{1}'.format(save_loc, save_file), 'rb') as f:
        d = pkl.load(f)
    bg_bs_npy2r_bin = d['npy2r_bin']
    bg_bs_tac2_bin = d['tac2_bin']
    bg_bs_car4_bin = d['car4_bin']
    gfk
    print('Data loaded')
    
except:
    print('No data found, calculating p values')

    no_cells = len(Cell.cell_data.keys())
    bg_bs_npy2r_bin = {}
    bg_bs_tac2_bin = {}
    bg_bs_car4_bin = {}

    local_px = int(local_region/um_per_px/2)
    min_dist_px = min_dist/um_per_px

    for cell in range(no_cells):
#     for cell in cells_sub:
        
        if np.mod(cell, 10) == 0:
            t1 = np.round(time.time() - t0)
            print('Cell {0}, {1} seconds'.format(cell, t1))

        cell_no = cell + 1
        cell_dict = Cell.cell_data[cell_no]
        z_planes = cell_dict['z_planes']

        bg_bs_npy2r_bin[cell] = {}
        bg_bs_tac2_bin[cell] = {}
        bg_bs_car4_bin[cell] = {}
        
        p = 0

        for plane in z_planes:

            bg_bs_npy2r_bin[cell][plane] = np.zeros(n_sample)
            bg_bs_tac2_bin[cell][plane] = np.zeros(n_sample)
            bg_bs_car4_bin[cell][plane] = np.zeros(n_sample)
            
            center = centers[cell][plane]
            n_verts = n_vert_all[cell][plane]

            x1 = np.max([0, center[0] - local_px])
            x2 = np.min([h, center[0] + local_px])
            y1 = np.max([0, center[1] - local_px])
            y2 = np.min([w, center[1] + local_px])

            center_local = [local_px, local_px]

            local_bg = bg_all[plane, x1:x2, y1:y2].astype(bool)

            verts = np.array(np.where(local_bg))
            dist_from_center = np.linalg.norm(verts - np.reshape(center_local, [2, 1]), 
                                              axis = 0)
            verts = verts[:, dist_from_center > min_dist_px]
            dist_from_center = dist_from_center[dist_from_center > min_dist_px]

            order = np.argsort(dist_from_center)
            n_verts_bg = np.min([bg_factor*n_verts, len(order)])

            verts = verts[:, order[0:n_verts_bg]]
            samples = np.random.choice(np.linspace(0, n_verts_bg - 1, n_verts_bg).astype(int), 
                                       (n_verts, n_sample), replace = True)
            x_bs = verts[0][samples]
            y_bs = verts[1][samples]

            local_im_npy2r = im_bin_npy2r[plane, x1:x2, y1:y2]
            local_im_tac2 = im_bin_tac2[plane, x1:x2, y1:y2]
            local_im_car4 = im_bin_car4[plane, x1:x2, y1:y2]


            local_bg_bs = np.zeros([x2 - x1, y2 - y1]).astype(bool)
            for s in range(n_sample):
                local_bg_bs[x_bs[:, s], y_bs[:, s]] = np.ones(n_verts)
                bg_bs_npy2r_bin[cell][plane][s] = np.mean(local_im_npy2r[local_bg_bs])
                bg_bs_car4_bin[cell][plane][s] = np.mean(local_im_car4[local_bg_bs])
                bg_bs_tac2_bin[cell][plane][s] = np.mean(local_im_tac2[local_bg_bs])

            p += 1
            
    d['npy2r_bin'] = bg_bs_npy2r_bin 
    d['tac2_bin'] = bg_bs_tac2_bin 
    d['car4_bin'] = bg_bs_car4_bin 

    with open('{0}\\{1}'.format(save_loc, save_file), 'wb') as f:
        pkl.dump(d, f)     
    

No data found, calculating p values
Cell 0, 0.0 seconds
Cell 10, 8.0 seconds
Cell 20, 21.0 seconds
Cell 30, 33.0 seconds
Cell 40, 43.0 seconds
Cell 50, 55.0 seconds
Cell 60, 66.0 seconds
Cell 70, 75.0 seconds
Cell 80, 85.0 seconds
Cell 90, 96.0 seconds


### Calculate p-value  and confidence interval

In [75]:
save_file4

'S3_pvals_lipo_excl_bin.pkl'

In [74]:
save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR 3.12\\S3'
save_file1 = 'S3_pvals.pkl'
save_file2 = 'S3_conf_intvls.pkl'
save_file3 = 'S3_pvals_lipo_excl.pkl'
save_file4 = 'S3_pvals_lipo_excl_bin.pkl'
save_file5 = 'S3_pvals_bin_all.pkl'

p_value = 0.05 # Threshold for calling signal significantly different from background

try:
    with open('{0}\\{1}'.format(save_loc, save_file1), 'rb') as f:
        p_vals = pkl.load(f)
        pv_npy2r = p_vals['npy2r']
        pv_tac2 = p_vals['tac2']
        pv_car4 = p_vals['car4']
    print('Loaded p values for raw images')
    
    with open('{0}\\{1}'.format(save_loc, save_file2), 'rb') as f:
        ci = pkl.load(f)
        ci_npy2r = ci['npy2r']
        ci_tac2 = ci['tac2']
        ci_car4 = ci['car4']
    print('Loaded confidence intervals')
    
    with open('{0}\\{1}'.format(save_loc, save_file3), 'rb') as f:
        p_vals = pkl.load(f)
        pv_npy2r2 = p_vals['npy2r']
        pv_tac22 = p_vals['tac2']
        pv_car42 = p_vals['car4']
    print('Loaded p values for lipofuscin excluded masks')
    
    with open('{0}\\{1}'.format(save_loc, save_file4), 'rb') as f:
        p_vals = pkl.load(f)
        pv_npy2r3 = p_vals['npy2r']
        pv_tac23 = p_vals['tac2']
        pv_car43 = p_vals['car4']
    print('Loaded p values for lipofuscin excluded masks in binarized images')
    
    with open('{0}\\{1}'.format(save_loc, save_file5), 'rb') as f:
        p_vals = pkl.load(f)
        pv_npy2r4 = p_vals['npy2r']
        pv_tac24 = p_vals['tac2']
        pv_car44 = p_vals['car4']
    print('Loaded p values for binarized images')
    
    fj

except:
    print('Data not found; calculating')
    
    ci_npy2r = np.ones(no_cells)
    ci_car4 = np.ones(no_cells)
    ci_tac2 = np.ones(no_cells)

    pv_npy2r = np.ones(no_cells)
    pv_car4 = np.ones(no_cells)
    pv_tac2 = np.ones(no_cells)
    
    pv_npy2r2 = np.ones(no_cells)
    pv_car42 = np.ones(no_cells)
    pv_tac22 = np.ones(no_cells)
    
    pv_npy2r3 = np.ones(no_cells)
    pv_car43 = np.ones(no_cells)
    pv_tac23 = np.ones(no_cells)
    
    pv_npy2r4 = np.ones(no_cells)
    pv_car44 = np.ones(no_cells)
    pv_tac24 = np.ones(no_cells)
    
    for cell in range(no_cells):
#     for cell in cells_sub:
        
        if np.mod(cell, 10) == 0:
            print('Cell {0}'.format(cell))

        cell_no = cell + 1
        cell_dict = Cell.cell_data[cell_no]
        z_planes = cell_dict['z_planes']

        # npy2r
        
        bs_means = bg_bs_npy2r[cell][z_planes[0]]
        for plane in z_planes[1:]:
            bs_means = np.concatenate([bs_means, bg_bs_npy2r[cell][plane]])
        pv_npy2r[cell] = np.sum(bs_means > avg_intensity_npy2r[cell])/len(bs_means)
        ci_npy2r[cell] = np.mean(np.percentile(bs_means, (1 - p_value)*100, axis = 0))
        pv_npy2r2[cell] = np.sum(bs_means > int_npy2r[cell])/len(bs_means)
        
        bs_means = bg_bs_npy2r_bin[cell][z_planes[0]]
        for plane in z_planes[1:]:
            bs_means = np.concatenate([bs_means, bg_bs_npy2r_bin[cell][plane]])
        pv_npy2r3[cell] = np.sum(bs_means >= int_npy2r_bin[cell])/len(bs_means)
        #pv_npy2r4[cell] = np.sum(bs_means >= int_npy2r_bin_all[cell])/len(bs_means)
        
        # car4
        
        bs_means = bg_bs_car4[cell][z_planes[0]]
        for plane in z_planes[1:]:
            bs_means = np.concatenate([bs_means, bg_bs_car4[cell][plane]])
        pv_car4[cell] = np.sum(bs_means > avg_intensity_car4[cell])/len(bs_means)
        pv_car42[cell] = np.sum(bs_means > int_car4[cell])/len(bs_means)
        ci_car4[cell] = np.mean(np.percentile(bs_means, (1 - p_value)*100, axis = 0))

        bs_means = bg_bs_car4_bin[cell][z_planes[0]]
        for plane in z_planes[1:]:
            bs_means = np.concatenate([bs_means, bg_bs_car4_bin[cell][plane]])
        pv_car43[cell] = np.sum(bs_means >= int_car4_bin[cell])/len(bs_means)
        #pv_car44[cell] = np.sum(bs_means >= int_car4_bin_all[cell])/len(bs_means)

        
        # tac2
        
        bs_means = bg_bs_tac2[cell][z_planes[0]]
        for plane in z_planes[1:]:
            bs_means = np.concatenate([bs_means, bg_bs_tac2[cell][plane]])
        pv_tac2[cell] = np.sum(bs_means > avg_intensity_tac2[cell])/len(bs_means)
        pv_tac22[cell] = np.sum(bs_means > int_tac2[cell])/len(bs_means)
        ci_tac2[cell] = np.mean(np.percentile(bs_means, (1 - p_value)*100, axis = 0))
        
        bs_means = bg_bs_tac2_bin[cell][z_planes[0]]
        for plane in z_planes[1:]:
            bs_means = np.concatenate([bs_means, bg_bs_tac2_bin[cell][plane]])
        pv_tac23[cell] = np.sum(bs_means >= int_tac2_bin[cell])/len(bs_means)
        #pv_tac24[cell] = np.sum(bs_means >= int_tac2_bin_all[cell])/len(bs_means)


    with open('{0}\\{1}'.format(save_loc, save_file1), 'wb') as f:
        pkl.dump({'npy2r': pv_npy2r, 'tac2': pv_tac2, 'car4':pv_car4}, f)

    with open('{0}\\{1}'.format(save_loc, save_file2), 'wb') as f:
        pkl.dump({'npy2r': ci_npy2r, 'tac2': ci_tac2, 'car4':ci_car4}, f)

    with open('{0}\\{1}'.format(save_loc, save_file3), 'wb') as f:
        pkl.dump({'npy2r': pv_npy2r2, 'tac2': pv_tac22, 'car4':pv_car42}, f)

    with open('{0}\\{1}'.format(save_loc, save_file4), 'wb') as f:
        pkl.dump({'npy2r': pv_npy2r3, 'tac2': pv_tac23, 'car4':pv_car43}, f)
        
    with open('{0}\\{1}'.format(save_loc, save_file5), 'wb') as f:
        pkl.dump({'npy2r': pv_npy2r4, 'tac2': pv_tac24, 'car4':pv_car44}, f)

    
print('')
print('With raw images:')

print('npy2r: {0} cells are significant'.format(np.sum(pv_npy2r < 0.05)))
print('car4: {0} cells are significant'.format(np.sum(pv_car4 < 0.05)))
print('tac2: {0} cells are significant'.format(np.sum(pv_tac2 < 0.05)))

print('')

print('npy2r & tac2: {0} cells are significant'.format(np.sum(np.logical_and(pv_npy2r < 0.05, pv_tac2 < 0.05))))
print('car4 & tac2: {0} cells are significant'.format(np.sum(np.logical_and(pv_car4 < 0.05, pv_tac2 < 0.05))))
print('npy2r & car4: {0} cells are significant'.format(np.sum(np.logical_and(pv_npy2r < 0.05, pv_car4 < 0.05))))

print('')
print('With lipofuscin pixels exluded:')

print('npy2r: {0} cells are significant'.format(np.sum(pv_npy2r2 < 0.05)))
print('car4: {0} cells are significant'.format(np.sum(pv_car42 < 0.05)))
print('tac2: {0} cells are significant'.format(np.sum(pv_tac22 < 0.05)))

print('')

print('npy2r & tac2: {0} cells are significant'.format(np.sum(np.logical_and(pv_npy2r2 < 0.05, pv_tac22 < 0.05))))
print('car4 & tac2: {0} cells are significant'.format(np.sum(np.logical_and(pv_car42 < 0.05, pv_tac22 < 0.05))))
print('npy2r & car4: {0} cells are significant'.format(np.sum(np.logical_and(pv_npy2r2 < 0.05, pv_car42 < 0.05))))

print('')
print('With lipofuscin pixels exluded, in binarized images:')

print('npy2r: {0} cells are significant'.format(np.sum(pv_npy2r3 < 0.05)))
print('car4: {0} cells are significant'.format(np.sum(pv_car43 < 0.05)))
print('tac2: {0} cells are significant'.format(np.sum(pv_tac23 < 0.05)))

print('')

print('npy2r & tac2: {0} cells are significant'.format(np.sum(np.logical_and(pv_npy2r3 < 0.05, pv_tac23 < 0.05))))
print('car4 & tac2: {0} cells are significant'.format(np.sum(np.logical_and(pv_car43 < 0.05, pv_tac23 < 0.05))))
print('npy2r & car4: {0} cells are significant'.format(np.sum(np.logical_and(pv_npy2r3 < 0.05, pv_car43 < 0.05))))

print('')
print('With all pixels, in binarized images:')

print('npy2r: {0} cells are significant'.format(np.sum(pv_npy2r4 < 0.05)))
print('car4: {0} cells are significant'.format(np.sum(pv_car44 < 0.05)))
print('tac2: {0} cells are significant'.format(np.sum(pv_tac24 < 0.05)))

print('')

print('npy2r & tac2: {0} cells are significant'.format(np.sum(np.logical_and(pv_npy2r4 < 0.05, pv_tac24 < 0.05))))
print('car4 & tac2: {0} cells are significant'.format(np.sum(np.logical_and(pv_car44 < 0.05, pv_tac24 < 0.05))))
print('npy2r & car4: {0} cells are significant'.format(np.sum(np.logical_and(pv_npy2r4 < 0.05, pv_car44 < 0.05))))

Loaded p values for raw images
Loaded confidence intervals
Loaded p values for lipofuscin excluded masks
Loaded p values for lipofuscin excluded masks in binarized images
Loaded p values for binarized images
Data not found; calculating
Cell 0
Cell 10
Cell 20
Cell 30
Cell 40
Cell 50
Cell 60
Cell 70
Cell 80
Cell 90

With raw images:
npy2r: 99 cells are significant
car4: 57 cells are significant
tac2: 99 cells are significant

npy2r & tac2: 98 cells are significant
car4 & tac2: 57 cells are significant
npy2r & car4: 57 cells are significant

With lipofuscin pixels exluded:
npy2r: 94 cells are significant
car4: 41 cells are significant
tac2: 97 cells are significant

npy2r & tac2: 91 cells are significant
car4 & tac2: 41 cells are significant
npy2r & car4: 40 cells are significant

With lipofuscin pixels exluded, in binarized images:
npy2r: 24 cells are significant
car4: 26 cells are significant
tac2: 13 cells are significant

npy2r & tac2: 7 cells are significant
car4 & tac2: 5 cells are 

## Check classification based on manual selection

In [45]:
# npy2r
#detected = pv_npy2r < 0.05
cells = list(range(Cell.n_cells))
detected = [i for i in cells if pv_npy2r3[i] < 0.05]
manual = npy2r_pos

true_pos = [i for i in detected if i in manual]
false_pos = [i for i in detected if not i in manual]
true_neg = [i for i in cells if np.logical_and(not i in detected, not i in manual)]
false_neg = [i for i in manual if not i in detected]

print('npy2r')
print('{0} true positives'.format(len(true_pos)))
print('{0} false positives'.format(len(false_pos)))
print('{0} true negative'.format(len(true_neg)))
print('{0} false negative'.format(len(false_neg)))

# car4
#detected = pv_car4 < 0.05
detected = [i for i in cells if pv_car43[i] < 0.05]
manual = car4_pos

true_pos = [i for i in detected if i in manual]
false_pos = [i for i in detected if not i in manual]
true_neg = [i for i in cells if np.logical_and(not i in detected, not i in manual)]
false_neg = [i for i in manual if not i in detected]

print('car4')
print('{0} true positives'.format(len(true_pos)))
print('{0} false positives'.format(len(false_pos)))
print('{0} true negative'.format(len(true_neg)))
print('{0} false negative'.format(len(false_neg)))


# tac2
#detected = pv_tac2 < 0.05
detected = [i for i in cells if pv_tac23[i] < 0.05]
manual = tac2_pos

true_pos = [i for i in detected if i in manual]
false_pos = [i for i in detected if not i in manual]
true_neg = [i for i in cells if np.logical_and(not i in detected, not i in manual)]
false_neg = [i for i in manual if not i in detected]

print('tac2')
print('{0} true positives'.format(len(true_pos)))
print('{0} false positives'.format(len(false_pos)))
print('{0} true negative'.format(len(true_neg)))
print('{0} false negative'.format(len(false_neg)))




Ndnf
74 true positives
7 false positives
230 true negative
12 false negative
Vip
35 true positives
16 false positives
270 true negative
2 false negative
Sst
13 true positives
0 false positives
299 true negative
11 false negative


# Find cortex surface and calculate depths

In [49]:
save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR 3.12\\S3'
save_file = 'S3_depths.pkl'

try:
    with open('{0}\\{1}'.format(save_loc, save_file), 'rb') as f:
        depths = pkl.load(f)
        print('Depths loaded')
except:
    print('Data not found, draw cortex surface and execute lower cell')

Depths loaded


In [68]:
viewer.add_shapes(data = None, name = 'surface')

<Shapes layer 'surface' at 0x198a22d1748>

In [76]:
um_per_px = 0.13
save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR 3.12\\S3'
save_file = 'S3_depths.pkl'

def find_depth(d1, d2, cell_center): #d1 and d2 are two points on the surface closest to the cell center
    
    dx = d2[0] - d1[0]
    dy = d2[1] - d1[1]
    d1_cell_x = cell_center[0] - d1[0]
    d1_cell_y = cell_center[1] - d1[1]
    t = (dx * d1_cell_x + dy * d1_cell_y) / (dx * dx + dy * dy)
    px = d1[0] + t * dx # Projection of cell center onto line joining d1 and d2
    py = d1[1] + t * dy
    depth = np.linalg.norm([px - cell_center[0], py - cell_center[1]])
    return depth

# Draw surface and calculate depths
surface = viewer.layers['surface'].data[-1][:, 1:]
depths = np.zeros(100)

for cell in range(100):
    pls = centers[cell].keys()
    cx = []
    cy = []
    for pl in pls:
        cx = np.append(cx, centers[cell][pl][0])
        cy = np.append(cy, centers[cell][pl][1])
    c = [np.mean(cx), np.mean(cy)]
    distances = np.linalg.norm(c - surface, axis = 1)
    order = np.argsort(distances)
    [p1, p2] = order[0:2]
    d1 = surface[p1]
    d2 = surface[p2]
    depths[cell] = find_depth(d1, d2, c)
depths = depths*um_per_px

with open('{0}\\{1}'.format(save_loc, save_file), 'wb') as f:
        pkl.dump(depths, f)


In [71]:
Cell.n_cells = 100

In [77]:
save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR 3.11\\S3'
save_file = 'S3_depths.pkl'

with open('{0}\\{1}'.format(save_loc, save_file), 'wb') as f:
        pkl.dump(depths, f)

# Plots

## 2D scatter plot of pixel values (pairs of channels)

In [87]:
#img_dict = {'npy2r': im_bin_npy2r, 'tac2': im_bin_tac2, 'car4': im_bin_car4}
img_dict = {'npy2r': im_diff_npy2r_lipo, 'tac2': im_diff_tac2_lipo, 'car4': im_diff_car4_lipo}
#img_dict = {'npy2r': im_array_npy2r, 'tac2': im_array_tac2, 'car4': im_array_car4}

pixels_dict = {'Cells': all_cell_pixels, 'Lipofuscin': all_lipo_pixels}
#pixels_dict = {'Cells': all_cell_pixels}

colors_dict = {'Cells': 'k', 'Lipofuscin': 'r'}

alpha_dict = {'Cells': 0.4, 'Lipofuscin': 0.4}

title = 'Diff gauss sigma = ({0}, {1}); binarized above {2} std from median'.format(sigma_small, sigma_large, thresh_scale)
#title = 'Raw images'
save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR_10.17\\S1_scatter_plots'
save_file = '{0} scatter lipo + cells.png'.format(title)
#save_file = '{0} scatter cells.png'.format(title)

scatter_channels.scatter_channels(img_dict, pixels_dict, max_per_group= 100000, colors_dict = colors_dict,
                                  alpha_dict = alpha_dict,
#                                   make_3D = True,
#                                   title = title, 
#                                   save = True, 
                                  save_loc = save_loc, save_file = save_file)

del img_dict
del pixels_dict

## 3D scatter plot of pixel values (3 channels)

In [23]:
plt.figure()
plt.plot(all_cell_pixels[:, 0])

In [183]:
# -*- coding: utf-8 -*-
"""
Created on Thu Feb 27 15:36:38 2020

Input:
    img_dict: Dictionary in which each key is the name of a channel and
              the corresponding value is the image of shape [n_planes, height, width] for that channel
    pixels_dict: Dictionary in which each key is the name of a group of pixels and
                 the corresponding value is an array of pixel coordinates of shape [n_pixels, 3]
    colors_dict: (optional) Dictionary in which each key is the name of a group of pixels and
                 the corresponding value is a color
    max_per_group: Default 10000. Maximum number of pixels per group to plot.
                   If number of pixels exceeds this, pixels will be randomly sub-sampled.


@author: Amrita Singh
"""

from itertools import combinations
import matplotlib.pyplot as plt
import numpy as np
from os.path import sep


def scatter_channels2(img_dict, pixels_dict, max_per_group = 10000, colors_dict = None,
                     make_3D = False, alpha_dict = None, 
                     title = None, save = False, save_loc = None, save_file = None, log_scale = False):


    # %%
    channel_names = list(img_dict.keys())
    n_channels = len(channel_names)
    channels = range(n_channels)

    if make_3D:
        combos = list(combinations(channels, 3))
    else:
        combos = list(combinations(channels, 2))

    n_combos = len(combos)

    px_groups = list(pixels_dict.keys())


    # %% Get values for each channel for each pixel group
    px_vals = {}

    for group in px_groups:

        px = pixels_dict[group]
        n_px = px.shape[0]

        # Sub-sample if too many pixels to plot
        if n_px > max_per_group:
            idx = np.random.randint(int(n_px/max_per_group), size = n_px)
            px = px[idx == 0, :]
            if type(colors_dict[group]) is np.ndarray:
                if colors_dict[group].shape[0] > max_per_group:
                    colors_dict[group] = colors_dict[group][idx == 0]

        px_vals[group] = {}

        for channel in channel_names:
            px_vals[group][channel] = img_dict[channel][(px[:, 0], px[:, 1], px[:, 2])]

    # %%
    fig, ax = plt.subplots(nrows = n_combos if n_combos > 1 else 2, ncols = 1, figsize = (20, 20))
    for combo in range(n_combos):

        if make_3D:
            c3 = channel_names[combos[combo][2]]
            ax[combo] = plt.axes(projection='3d')
            ax[combo].set_zlabel(c3, fontsize = 18)

        c1 = channel_names[combos[combo][0]]
        c2 = channel_names[combos[combo][1]]

        ax[combo].set_xlabel(c1, fontsize = 18)
        ax[combo].set_ylabel(c2, fontsize = 18)



        for group in px_groups:

            if log_scale:
                ax[combo].set_xscale('log')
                ax[combo].set_yscale('log')
                if make_3D:
                    ax[combo].set_zscale('log')

            if colors_dict == None:
                if make_3D:
                    p = ax[combo].scatter3D(px_vals[group][c1], px_vals[group][c2], px_vals[group][c3], label = group,
                      marker = '.', alpha = alpha_dict[group])
                    fig.colorbar(p)
                else:
                    ax[combo].scatter(px_vals[group][c1], px_vals[group][c2], label = group, marker = '.', alpha = alpha_dict[group])
            else:
                if make_3D:
                    ax[combo].scatter3D(px_vals[group][c1], px_vals[group][c2], px_vals[group][c3], label = group,
                      marker = '.', alpha = alpha_dict[group], c = colors_dict[group], cmap = 'bwr')
                else:
                    ax[combo].scatter(px_vals[group][c1], px_vals[group][c2], label = group, c = colors_dict[group], marker = '.', alpha = alpha_dict[group])


        ax[combo].legend(fontsize = 18)


    if not title == None:
        ax[0].set_title(title, fontsize = 18)

    if save:
        if save_loc == None:
            print('Please provide save location')
        else:
            if save_file == None:
                print('Please provide save file name')
            else:
                plt.savefig('{0}{1}{2}'.format(save_loc, sep, save_file))


In [211]:
#img_dict = {'npy2r': im_bin_npy2r, 'vip': im_bin_vip, 'car4': im_bin_car4}
img_dict = {'546': im_bin_npy2r_lipo, '647': im_bin_tac2_lipo, '594': im_bin_car4_lipo}
#img_dict = {'npy2r': im_diff_npy2r_lipo, 'tac2': im_diff_tac2_lipo, 'car4': im_diff_car4_lipo}
#img_dict = {'npy2r': im_bin_npy2r, 'tac2': im_bin_tac2, 'car4': im_bin_car4}
#img_dict = {'546': im_array_npy2r, '647': im_array_tac2, '594': im_array_car4}
#img_dict = {'546': im_npy2r_lipo, '647': im_tac2_lipo, '594': im_car4_lipo}


#pixels_dict = {'Cells': all_cell_pixels, 'Lipofuscin': lipo_pixels}
pixels_dict = {'Cells': all_cell_pixels}
#pixels_dict = {'Cells': lipo_pixels, 'Lipofuscin': all_lipo_pixels}
#pixels_dict = {'Lipofuscin': all_lipo_pixels}


#colors_dict = {'Cells': 'k', 'Lipofuscin': 'b'}
#colors_dict = {'Cells': m_dist_vals['Cells'], 'Lipofuscin': m_dist_vals['Lipofuscin']}
#colors_dict = {'Cells': np.log(m_dist_vals['Cells']), 'Lipofuscin': 'r'}
#colors_dict ={'Cells': np.logical_not(m_dist_vals['Cells'] > thresh), 'Lipofuscin': 'r'}
#colors_dict = {'Cells': np.logical_not(np.log(mean_vals) < 5), 'Lipofuscin': 'r'}
#colors_dict = {'Cells': np.logical_not([a in lipo_pixels for a in all_cell_pixels])}
colors_dict = {'Cells': cell_px_is_lipo}

alpha_dict = {'Cells': 0.4, 'Lipofuscin': 1}

#title = 'Sub-sampled diff gauss sigma = ({0}, {1}) \n binarized above {2} std from median \n color = log mahalanobis distance from lipofuscin pixels'.format(sigma_small, sigma_large, thresh_scale)
#title = 'Raw images'

save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR 3.12\\S3\\Plots'
#save_file = '3D scatter lipo + cells mahalanobis distance2.png'
#save_file = '{0} 3D scatter cells + lipo.png'.format(title)
#save_file = '{0} 3D scatter lipo.png'.format(title)
save_file=  '3D scatter lipo + cells.png'

scatter_channels2(img_dict, pixels_dict, max_per_group= 100000, colors_dict = colors_dict,
                                  alpha_dict = alpha_dict,
                                  make_3D = True,
#                                   title = title, 
#                                   save = True, 
                                  save_loc = save_loc, save_file = save_file)

del img_dict
del pixels_dict

In [210]:
t0 = time.time()
n = len(all_cell_pixels)
cell_px_is_lipo = np.zeros(n)
for px in range(n):
    pc = all_cell_pixels[px]
    if im_array_lipo[pc[0], pc[1], pc[2]] == 1:
        cell_px_is_lipo[px] = 1

In [47]:
np.max(np.log(m_dist_vals['Cells']))

5.031541064199322

In [112]:
np.log(thresh)

0.64019138924129

In [198]:
plt.close('all')

#### Colorbar

In [182]:
plt.figure()
a = np.zeros([4, 4])
a[0, 0] = np.max(np.log(m_dist_vals['Cells']))
a[0, 1] = np.min(np.log(m_dist_vals['Cells']))
# a[0, 0] = np.max(np.log(mean_vals))
# a[0, 1] = np.min(np.log(mean_vals))
plt.imshow(a)
plt.colorbar(label = 'Log of mahalanobis distance from lipofuscin pixels')

### Simple

In [30]:
ax = plt.axes(projection = '3d')
ax.scatter3D()



## Histogram of pixel values in each channel

In [25]:
#img_dict = {'npy2r': im_diff_npy2r, 'tac2': im_diff_tac2, 'car4': im_diff_car4}
img_dict = {'npy2r': im_array_npy2r, 'tac2': im_array_tac2, 'car4': im_array_car4}

pixels_dict = {'Cells': all_cell_pixels, 'Lipofuscin': all_lipo_pixels}
#pixels_dict = {'Cells': all_cell_pixels}
colors_dict = {'Cells': 'b', 'Lipofuscin': 'k'}
#title = 'Diff gauss sigma = ({0}, {1})'.format(sigma_small, sigma_large)
title = 'Raw images'
save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR_10.17\\S1_scatter_plots'
save_file = '{0} hist lipo + cells.png'.format(title)
#save_file = '{0} hist cells.png'.format(title)

hist_channels.hist_channels(img_dict, pixels_dict, max_per_group= 100000, colors_dict = colors_dict, 
                            title = title, save = True, save_loc = save_loc, save_file = save_file)

del img_dict
del pixels_dict

## Signal vs depth

### Using p-value for significance

In [73]:
# Plot intensity vs depth for all channels
fig, ax = plt.subplots(nrows = 3, ncols = 3)
    
d = depths

npos = pv_npy2r < 0.05
vpos = pv_car4 < 0.05
spos = pv_tac2 < 0.05

n = avg_intensity_npy2r
v = avg_intensity_car4
s = avg_intensity_tac2

bn = avg_bg_npy2r
bv = avg_bg_car4
bs = avg_bg_tac2

ax[0, 0].scatter(d, n - bn, color = 'b', alpha = 0.1, marker = '.')
ax[0, 0].scatter(d[npos], n[npos] - bn[npos], color = 'b', marker = '.')
#ax[0, 0].set_xlabel('Depth from pia (um)')
ax[0, 0].set_ylabel('Intensity in channel (bold => p < 0.05)')
ax[0, 0].set_title('npy2r: all pixels')

ax[0, 1].scatter(d, v - bv, color = 'r', alpha = 0.1, marker = '.')
ax[0, 1].scatter(d[vpos], v[vpos] - bv[vpos], color = 'r', marker = '.')
#ax[0, 1].set_xlabel('Depth from pia (um)')
ax[0, 1].set_title('car4: all pixels')

ax[0, 2].scatter(d, s - bs, color = 'g', alpha = 0.1, marker = '.')
ax[0, 2].scatter(d[spos], s[spos] - bs[spos], color = 'g', marker = '.')
#ax[0, 2].set_xlabel('Depth from pia (um)')
ax[0, 2].set_title('tac2: all pixels')

n = int_npy2r
v = int_car4
s = int_tac2

npos2 = pv_npy2r2 < 0.05
vpos2 = pv_car42 < 0.05
spos2 = pv_tac22 < 0.05

ax[1, 0].scatter(d, n - bn, color = 'b', alpha = 0.1, marker = '.')
ax[1, 0].scatter(d[npos2], n[npos2] - bn[npos2], color = 'b', marker = '.')
#ax[1, 0].set_xlabel('Depth from pia (um)')
ax[1, 0].set_ylabel('Intensity in channel (bold => p < 0.05)')
ax[1, 0].set_title('npy2r: lipofuscin pixels excluded')

ax[1, 1].scatter(d, v - bv, color = 'r', alpha = 0.1, marker = '.')
ax[1, 1].scatter(d[vpos2], v[vpos2] - bv[vpos2], color = 'r', marker = '.')
#ax[1, 1].set_xlabel('Depth from pia (um)')
ax[1, 1].set_title('car4: lipofuscin pixels excluded')

ax[1, 2].scatter(d, s - bs, color = 'g', alpha = 0.1, marker = '.')
ax[1, 2].scatter(d[spos2], s[spos2] - bs[spos2], color = 'g', marker = '.')
#ax[1, 2].set_xlabel('Depth from pia (um)')
ax[1, 2].set_title('tac2: lipofuscin pixels excluded')

n = int_npy2r_bin
v = int_car4_bin
s = int_tac2_bin

bn = avg_bg_npy2r_bin
bv = avg_bg_car4_bin
bs = avg_bg_tac2_bin

npos3 = pv_npy2r3 < 0.05
vpos3 = pv_car43 < 0.05
spos3 = pv_tac23 < 0.05

# vpos3 = int_car4_bin - avg_bg_car4_bin > 200
# spos3 = int_tac2_bin - avg_bg_tac2_bin > 500


ax[2, 0].scatter(d, n - bn, color = 'b', alpha = 0.1, marker = '.')
ax[2, 0].scatter(d[npos3], n[npos3] - bn[npos3], color = 'b', marker = '.')
ax[2, 0].set_xlabel('Depth from pia (um)')
ax[2, 0].set_ylabel('Intensity in channel (bold => p < 0.05)')
ax[2, 0].set_title('npy2r: lipofuscin pixels excluded; Binarized images')

ax[2, 1].scatter(d, v - bv, color = 'r', alpha = 0.1, marker = '.')
ax[2, 1].scatter(d[vpos3], v[vpos3] - bv[vpos3], color = 'r', marker = '.')
ax[2, 1].set_xlabel('Depth from pia (um)')
ax[2, 1].set_title('car4: lipofuscin pixels excluded; Binarized images')

ax[2, 2].scatter(d, s - bs, color = 'g', alpha = 0.1, marker = '.')
ax[2, 2].scatter(d[spos3], s[spos3] - bs[spos3], color = 'g', marker = '.')
ax[2, 2].set_xlabel('Depth from pia (um)')
ax[2, 2].set_title('tac2: lipofuscin pixels excluded; Binarized images')


ValueError: x and y must be the same size

### Using 95% confidence interval for significance

In [100]:
# Plot intensity vs depth for all channels
fig, ax = plt.subplots(nrows = 2, ncols = 3)
    
d = depths

n = avg_intensity_npy2r
v = avg_intensity_car4
s = avg_intensity_tac2

bn = avg_bg_npy2r
bv = avg_bg_car4
bs = avg_bg_tac2

npos = n - bn > ci_npy2r
vpos = v - bv > ci_car4
spos = s - bs > ci_tac2

ax[0, 0].scatter(d, n - bn, color = 'b', alpha = 0.1, marker = '.')
ax[0, 0].scatter(d[npos], n[npos] - bn[npos], color = 'b', marker = '.')
ax[0, 0].set_xlabel('Depth from pia (um)')
ax[0, 0].set_ylabel('Intensity in channel (bold => 95% CI)')
ax[0, 0].set_title('npy2r: all pixels')

ax[0, 1].scatter(d, v - bv, color = 'r', alpha = 0.1, marker = '.')
ax[0, 1].scatter(d[vpos], v[vpos] - bv[vpos], color = 'r', marker = '.')
ax[0, 1].set_xlabel('Depth from pia (um)')
ax[0, 1].set_title('car4: all pixels')

ax[0, 2].scatter(d, s - bs, color = 'g', alpha = 0.1, marker = '.')
ax[0, 2].scatter(d[spos], s[spos] - bs[spos], color = 'g', marker = '.')
ax[0, 2].set_xlabel('Depth from pia (um)')
ax[0, 2].set_title('tac2: all pixels')

n = int_npy2r
v = int_car4
s = int_tac2

npos = n - bn > ci_npy2r
vpos = v - bv > ci_car4
spos = s - bs > ci_tac2

ax[1, 0].scatter(d, n - bn, color = 'b', alpha = 0.1, marker = '.')
ax[1, 0].scatter(d[npos2], n[npos2] - bn[npos2], color = 'b', marker = '.')
ax[1, 0].set_xlabel('Depth from pia (um)')
ax[1, 0].set_ylabel('Intensity in channel (bold => 95% CI)')
ax[1, 0].set_title('npy2r: lipofuscin pixels excluded')

ax[1, 1].scatter(d, v - bv, color = 'r', alpha = 0.1, marker = '.')
ax[1, 1].scatter(d[vpos2], v[vpos2] - bv[vpos2], color = 'r', marker = '.')
ax[1, 1].set_xlabel('Depth from pia (um)')
ax[1, 1].set_title('car4: lipofuscin pixels excluded')

ax[1, 2].scatter(d, s - bs, color = 'g', alpha = 0.1, marker = '.')
ax[1, 2].scatter(d[spos2], s[spos2] - bs[spos2], color = 'g', marker = '.')
ax[1, 2].set_xlabel('Depth from pia (um)')
ax[1, 2].set_title('tac2: lipofuscin pixels excluded')


Text(0.5, 1.0, 'Sst: lipofuscin pixels excluded')

## Raw images of each cell

### Function for plotting

In [78]:
def view_all_cells(n_cells, n_rows, n_cols, signal, im_array, cell_data, size_um, um_per_px, h, w, 
                  save_loc, title, max_intensity = None, spine_color_pos = 'r', pos_cells = [], spine_width = 2,
                  show_masks = False):

    cell_ids = np.array(list(cell_data.keys()))
    cell_order = cell_ids[np.flip(np.argsort(signal)).astype(int)]
    
    size_px = int(size_um/um_per_px/2)
    mask_dil =  np.zeros([h, w]).astype(bool)
    
    if max_intensity == None:
        max_intensity = np.max(signal)
    min_intensity = np.min(im_array)

    t0 = time.time()
    fig, ax = plt.subplots(nrows = n_rows, ncols = n_cols, figsize = (30, 20))

    for cell in range(n_cells):   

        if np.mod(cell, 10) == 0:
            print('Cell {0}: {1} seconds'.format(cell, np.round(time.time() - t0)))

        row = int(cell/n_cols)
        col = np.mod(cell, n_cols)

        cell_id = cell_order[cell]

        planes = cell_data[cell_id]['z_planes']
        med_plane = int(np.median(planes))

        mask = cell_data[cell_id]['masks'][med_plane]
        [x_center, y_center] = np.mean(mask, 0).astype(int)
        [x1, y1] = [np.max([0, x_center - size_px]), np.max([0, y_center - size_px])]
        [x2, y2] = [np.min([h, x_center + size_px]), np.min([w, y_center + size_px])]
        mask_dil[x1:x2, y1:y2] = np.ones([x2 - x1, y2 - y1])

        small_im = np.reshape(im_array[med_plane, mask_dil], [np.min([x2 - x1, size_px*2]), np.min([y2 - y1, size_px*2])])
        ax[row, col].imshow(small_im, vmin = min_intensity, vmax = max_intensity)

        mask_dil[x1:x2, y1:y2] = np.zeros([x2 - x1, y2 - y1])
        
        if show_masks:
            
            cell_px = cell_pixels_no_lipo[cell_id][med_plane]
            cell_mask = np.zeros(small_im.shape)
            x = cell_px[0, :] - x1
            y = cell_px[1, :] - y1
#             for i in range(len(x)):
#                 cell_mask[x[i], y[i]] = 1
            cell_mask[x, y] = np.ones(len(x))
            ax[row, col].imshow(cell_mask, vmin = 0, vmax = 1, cmap = 'gray', alpha = 0.2)
            

#         ax[row, col].axis('off')
        if col == 0:
            ax[row, col].set_ylabel('{0}'.format(np.round(signal[cell], 2)))
#             ax[row, col].axis('on')
        
        if len(pos_cells) > 0:
            if pos_cells[cell] == 1:
                ax[row, col].axis('on')
                ax[row, col].spines['bottom'].set_color(spine_color_pos)
                ax[row, col].spines['top'].set_color(spine_color_pos)
                ax[row, col].spines['left'].set_color(spine_color_pos)
                ax[row, col].spines['right'].set_color(spine_color_pos)
                ax[row, col].spines['bottom'].set_linewidth(spine_width)
                ax[row, col].spines['top'].set_linewidth(spine_width)
                ax[row, col].spines['left'].set_linewidth(spine_width)
                ax[row, col].spines['right'].set_linewidth(spine_width)
        
        ax[row, col].tick_params(axis = 'both', which = 'both', bottom = False, top = False, left = False, 
                                 right = False, labeltop = False, labelright = False, labelleft = False,
                                labelbottom = False)
                
                
#         plt.tick_params(
#     axis='x',          # changes apply to the x-axis
#     which='both',      # both major and minor ticks are affected
#     bottom=False,      # ticks along the bottom edge are off
#     top=False,         # ticks along the top edge are off
#     labelbottom=False) # labels along the bottom edge are off

    
    plt.savefig('{0}{1}{2}.png'.format(save_loc, sep, title))


### Get centers and median planes

In [79]:
centers_med_planes = np.zeros([Cell.n_cells, 2])
med_planes = np.zeros(Cell.n_cells)

for cell in range(Cell.n_cells):
    if np.mod(cell, 10) == 0:
        print(cell)
    cell_id = cell + 1
    planes = Cell.cell_data[cell_id]['z_planes']
    med_planes[cell] = int(np.median(planes))
    
    mask = Cell.cell_data[cell_id]['masks'][med_planes[cell]]
    centers_med_planes[cell, :] = np.mean(mask, 0).astype(int)

0
10
20
30
40
50
60
70
80
90


### Run for different conditions

In [82]:
save_loc = 'G:{0}Shared drives{0}as_share{0}HCR{0}HCR 3.12{0}S3{0}Plots'.format(sep)

titles = {
            1: 'npy2r_raw_images_all_cells2',
#           2: 'npy2r_diff_images_all_cells',
#           3: 'npy2r_bin_images_all_cells2',
          4: 'tac2_raw_images_all_cells2',
#           5: 'tac2_diff_images_all_cells',
#           6: 'tac2_bin_images_all_cells2',
          7: 'car4_raw_images_all_cells2',
#           8: 'car4_diff_images_all_cells',
#           9: 'car4_bin_images_all_cells2',
         }

im_arrays = {
             1: im_array_npy2r,
#              2: im_diff_npy2r,
#              3: im_bin_npy2r,
             4: im_array_tac2,
#              5: im_diff_tac2,
#              6: im_bin_tac2,
             7: im_array_car4,
#              8: im_diff_car4,
#              9: im_bin_car4,
            }
             
signals = {
             1: int_npy2r_bin - avg_bg_npy2r_bin,
#            1: avg_intensity_npy2r - avg_bg_npy2r,
#            2: int_npy2r_diff - avg_bg_npy2r_diff,
#            3: int_npy2r_bin[cells] - avg_bg_npy2r_bin[cells],
           4: int_tac2_bin - avg_bg_tac2_bin,
#             4: avg_intensity_tac2 - avg_bg_tac2,
#            5: int_tac2_diff - avg_bg_tac2_diff,
#            6: int_tac2_bin[cells] - avg_bg_tac2_bin[cells],
            7: int_car4_bin - avg_bg_car4_bin,
#            7: avg_intensity_car4 - avg_bg_car4,
#            8: int_car4_diff - avg_bg_car4_diff,
#            9: int_car4_bin[cells] - avg_bg_car4_bin[cells],
          }

pos = {
        1: pv_npy2r3 < 0.05,
        3: pv_npy2r3 < 0.05,
        4: pv_tac23 < 0.05,
        6: pv_tac23 < 0.05,
        7: pv_car43 < 0.05,
        9: pv_car43 < 0.05
    
}

n_rows = 10
n_cols = 33

size_um = 30 # In um
um_per_px = 0.13 # Size of a pixel in um

conds = [1, 4, 7]
#conds = [3, 6, 9]
order = {}

#c_data = {cells[i] + 1: Cell.cell_data[cells[i] + 1] for i in range(len(cells))}
c_data = Cell.cell_data

for cond in conds:
    print(titles[cond])
    view_all_cells(Cell.n_cells, n_rows, n_cols, signals[cond], im_arrays[cond],
                   c_data, size_um, um_per_px,
                    h, w, save_loc, titles[cond], 
                   max_intensity = np.max(avg_intensity_npy2r - avg_bg_npy2r_bin),
                  pos_cells = pos[cond],
                  spine_color_pos = 'r',
                  spine_width = 5, show_masks = True)
    
    




npy2r_raw_images_all_cells2
Cell 0: 4.0 seconds
Cell 10: 5.0 seconds
Cell 20: 7.0 seconds


IndexError: index 3180 is out of bounds for axis 0 with size 230

### Select positive and negative cells for each gene

In [37]:
save_loc = 'G:{0}Shared drives{0}as_share{0}HCR{0}HCR 3.12{0}S3'.format(sep)
save_file = 'npy2r_tac2_car4_pos_cells.pkl'

try:
    with open('{0}{1}{2}'.format(save_loc, sep, save_file), 'rb') as f:
        pos = pkl.load(f)
        npy2r_pos = pos['npy2r_pos']
        tac2_pos = pos['tac2_pos']
        car4_pos = pos['car4_pos']
        print('Data loaded')
except:
    # npy2r
    ind = np.flip(np.argsort(signals[1]))
    npy2r_pos = ind[0:78]
    npy2r_pos = np.concatenate([npy2r_pos, [ind[i] for i in [80, 82, 85, 87, 89, 90, 113, 117]]])

    # tac2
    ind = np.flip(np.argsort(signals[4]))
    tac2_pos = np.array(ind[0:24])

    # car4
    ind = np.flip(np.argsort(signals[7]))
    car4_pos = np.array(ind[0:37])

    with open('{0}{1}{2}'.format(save_loc, sep, save_file), 'wb') as f:
        pkl.dump({'npy2r_pos': npy2r_pos, 'tac2_pos': tac2_pos, 'car4_pos': car4_pos}, f)


Data loaded


#### Check exclusivity

In [5]:
npy2r_car4 = [i for i in npy2r_pos if i in car4_pos]
npy2r_tac2 = [i for i in npy2r_pos if i in tac2_pos]
car4_tac2 = [i for i in car4_pos if i in tac2_pos]
print(npy2r_car4)
print(npy2r_tac2)
print(car4_tac2)

[]
[]
[]


#### Get planes of positive cells

In [69]:
planes = []
npy2r_pos2 = []
car4_pos2 = []
tac2_pos2 = []
for cell in npy2r_pos:
    cell_no = cell + 1
    cp = Cell.cell_data[cell_no]['z_planes']
    planes = np.append(planes, cp)
    if any([0 in cp, 1 in cp, 2 in cp, 3 in cp]):
        npy2r_pos2 = np.append(npy2r_pos2, cell)
    
for cell in car4_pos:
    cell_no = cell + 1
    cp = Cell.cell_data[cell_no]['z_planes']
    planes = np.append(planes, cp)
    if any([0 in cp, 1 in cp, 2 in cp, 3 in cp]):
        car4_pos2 = np.append(car4_pos2, cell)
        
for cell in tac2_pos:
    cell_no = cell + 1
    cp = Cell.cell_data[cell_no]['z_planes']
    planes = np.append(planes, cp)
    if any([0 in cp, 1 in cp, 2 in cp, 3 in cp]):
        tac2_pos2 = np.append(tac2_pos2, cell)

In [70]:
tac2_pos2

array([ 96., 127.,  92.,  76., 115.,  83.,   7.])

In [71]:
npy2r_pos2

array([ 71.,   2.,  29.,   0.,   3.,   1.,  95., 139.,  13.,  25.,  15.,
        66., 125.,  19.,  60.,  16.,   4.,  14., 109.,  72., 108.,  27.,
       154.,  68.,  39.,  73.,  53.,  54.,  42.,  18.,  56., 124.,  55.,
        52.,  32.,   5.,  97.,  40.,  64.])

# View individual cells

## npy2r+ cells

In [50]:
sig = int_npy2r_bin - avg_bg_npy2r_bin
max_sig = 10
all_cells = np.array(list(Cell.cell_data.keys())) - 1
cells = all_cells[np.logical_and(sig < max_sig, pv_npy2r3 < 0.05)]

for cell in cells:
    print('Cell {0}: depth {1}um, signal = {2}, p value = {3}'.format(cell, 
                                                                      np.round(depths[cell], decimals = 2), 
                                                                      np.round(sig[cell], decimals = 2), 
                                                                      np.round(pv_npy2r3[cell], decimals = 2)))
    

Cell 5: depth 64.87um, signal = 1.18, p value = 0.0
Cell 6: depth 135.64um, signal = 0.23, p value = 0.0
Cell 67: depth 50.28um, signal = 0.21, p value = 0.0
Cell 97: depth 137.39um, signal = 2.01, p value = 0.0
Cell 136: depth 148.21um, signal = 6.63, p value = 0.0
Cell 137: depth 159.82um, signal = 0.65, p value = 0.0
Cell 145: depth 19.2um, signal = 6.05, p value = 0.0
Cell 151: depth 138.32um, signal = 9.89, p value = 0.0
Cell 173: depth 20.77um, signal = 2.66, p value = 0.0
Cell 174: depth 26.42um, signal = 0.3, p value = 0.0
Cell 179: depth 57.61um, signal = 2.81, p value = 0.0
Cell 180: depth 46.21um, signal = 6.92, p value = 0.0
Cell 184: depth 65.9um, signal = 8.79, p value = 0.0
Cell 187: depth 169.83um, signal = 0.23, p value = 0.0
Cell 197: depth 288.41um, signal = 2.92, p value = 0.0
Cell 210: depth 74.62um, signal = 0.84, p value = 0.0
Cell 216: depth 252.5um, signal = 3.47, p value = 0.0
Cell 225: depth 67.66um, signal = 0.32, p value = 0.0
Cell 261: depth 53.6um, signal

## npy2r- cells

In [51]:
sig = int_npy2r_bin - avg_bg_npy2r_bin
max_sig = 0
all_cells = np.array(list(Cell.cell_data.keys())) - 1
cells = all_cells[np.logical_and(sig < max_sig, pv_npy2r3 > 0.05)]
cells = cells[np.random.randint(0, high = len(cells) - 1, size = 10)]
for cell in cells:
    print('Cell {0}: depth {1}um, signal = {2}, p value = {3}'.format(cell, 
                                                                      np.round(depths[cell], decimals = 2), 
                                                                      np.round(sig[cell], decimals = 2), 
                                                                      np.round(pv_npy2r3[cell], decimals = 2)))


Cell 241: depth 295.46um, signal = -0.78, p value = 1.0
Cell 253: depth 257.9um, signal = -0.7, p value = 1.0
Cell 22: depth 202.62um, signal = -1.19, p value = 1.0
Cell 253: depth 257.9um, signal = -0.7, p value = 1.0
Cell 22: depth 202.62um, signal = -1.19, p value = 1.0
Cell 101: depth 260.03um, signal = -0.42, p value = 1.0
Cell 22: depth 202.62um, signal = -1.19, p value = 1.0
Cell 241: depth 295.46um, signal = -0.78, p value = 1.0
Cell 155: depth 158.41um, signal = -4.55, p value = 1.0
Cell 101: depth 260.03um, signal = -0.42, p value = 1.0


In [55]:
sig = int_npy2r_bin - avg_bg_npy2r_bin
max_sig = 10
all_cells = np.array(list(Cell.cell_data.keys())) - 1
cells = all_cells[np.logical_and(sig < max_sig, pv_npy2r3 > 0.05)]
cells = cells[np.random.randint(0, high = len(cells) - 1, size = 10)]
for cell in cells:
    print('Cell {0}: depth {1}um, signal = {2}, p value = {3}'.format(cell, 
                                                                      np.round(depths[cell], decimals = 2), 
                                                                      np.round(sig[cell], decimals = 2), 
                                                                      np.round(pv_npy2r3[cell], decimals = 2)))


Cell 62: depth 147.15um, signal = 0.0, p value = 1.0
Cell 252: depth 276.69um, signal = 0.0, p value = 1.0
Cell 86: depth 299.18um, signal = 0.0, p value = 1.0
Cell 61: depth 127.6um, signal = 0.0, p value = 1.0
Cell 158: depth 145.01um, signal = 0.0, p value = 1.0
Cell 256: depth 334.67um, signal = 0.0, p value = 1.0
Cell 77: depth 164.86um, signal = 0.0, p value = 1.0
Cell 20: depth 150.32um, signal = 0.0, p value = 1.0
Cell 63: depth 182.25um, signal = 0.0, p value = 1.0
Cell 242: depth 394.36um, signal = 0.0, p value = 1.0


In [66]:
sig = int_npy2r_bin - avg_bg_npy2r_bin
min_sig = 0
all_cells = np.array(list(Cell.cell_data.keys())) - 1
cells = all_cells[np.logical_and(sig > min_sig, pv_npy2r3 > 0.05)]
cells = cells[np.random.randint(0, high = len(cells) - 1, size = 10)]
for cell in cells:
    print('Cell {0}: depth {1}um, signal = {2}, p value = {3}'.format(cell, 
                                                                      np.round(depths[cell], decimals = 2), 
                                                                      np.round(sig[cell], decimals = 2), 
                                                                      np.round(pv_npy2r3[cell], decimals = 2)))


Cell 54: depth 59.12um, signal = 0.5, p value = 0.33
Cell 54: depth 59.12um, signal = 0.5, p value = 0.33
Cell 54: depth 59.12um, signal = 0.5, p value = 0.33
Cell 54: depth 59.12um, signal = 0.5, p value = 0.33
Cell 54: depth 59.12um, signal = 0.5, p value = 0.33
Cell 54: depth 59.12um, signal = 0.5, p value = 0.33
Cell 54: depth 59.12um, signal = 0.5, p value = 0.33
Cell 54: depth 59.12um, signal = 0.5, p value = 0.33
Cell 54: depth 59.12um, signal = 0.5, p value = 0.33
Cell 54: depth 59.12um, signal = 0.5, p value = 0.33


## car4+ cells

In [52]:
sig = int_car4_bin - avg_bg_car4_bin
pv = pv_car43
max_sig = 10
all_cells = np.array(list(Cell.cell_data.keys())) - 1
cells = all_cells[np.logical_and(sig < max_sig, pv < 0.05)]
# cells = cells[np.random.randint(0, high = len(cells) - 1, size = 10)]
for cell in cells:
    print('Cell {0}: depth {1}um, signal = {2}, p value = {3}'.format(cell, 
                                                                      np.round(depths[cell], decimals = 2), 
                                                                      np.round(sig[cell], decimals = 2), 
                                                                      np.round(pv[cell], decimals = 2)))


Cell 34: depth 196.2um, signal = 0.24, p value = 0.0
Cell 67: depth 50.28um, signal = 0.25, p value = 0.0
Cell 69: depth 70.63um, signal = 6.82, p value = 0.0
Cell 75: depth 143.21um, signal = 0.65, p value = 0.0
Cell 79: depth 163.33um, signal = 8.56, p value = 0.0
Cell 101: depth 260.03um, signal = 0.6, p value = 0.0
Cell 114: depth 113.27um, signal = 1.55, p value = 0.0
Cell 117: depth 234.87um, signal = -1.09, p value = 0.0
Cell 118: depth 208.14um, signal = -1.23, p value = 0.0
Cell 165: depth 0.17um, signal = -0.11, p value = 0.0
Cell 187: depth 169.83um, signal = 0.26, p value = 0.03
Cell 198: depth 205.59um, signal = 6.32, p value = 0.0
Cell 227: depth 148.93um, signal = 0.71, p value = 0.0
Cell 230: depth 185.46um, signal = 8.54, p value = 0.0
Cell 244: depth 150.31um, signal = 4.42, p value = 0.0
Cell 245: depth 26.22um, signal = 1.31, p value = 0.0
Cell 278: depth 141.81um, signal = 1.0, p value = 0.0
Cell 313: depth 25.12um, signal = 4.69, p value = 0.0


## car4- cells

In [53]:
sig = int_car4_bin - avg_bg_car4_bin
pv = pv_car43
max_sig = 10
all_cells = np.array(list(Cell.cell_data.keys())) - 1

cells = all_cells[np.logical_and(sig < max_sig, pv > 0.05)]
cells = cells[np.random.randint(0, high = len(cells) - 1, size = 10)]
for cell in cells:
    print('Cell {0}: depth {1}um, signal = {2}, p value = {3}'.format(cell, 
                                                                      np.round(depths[cell], decimals = 2), 
                                                                      np.round(sig[cell], decimals = 2), 
                                                                      np.round(pv[cell], decimals = 2)))


Cell 174: depth 26.42um, signal = 0.0, p value = 1.0
Cell 23: depth 56.87um, signal = 0.0, p value = 1.0
Cell 31: depth 125.75um, signal = 0.0, p value = 1.0
Cell 92: depth 188.51um, signal = 0.0, p value = 1.0
Cell 94: depth 57.07um, signal = 0.0, p value = 1.0
Cell 260: depth 90.81um, signal = 0.0, p value = 1.0
Cell 132: depth 148.87um, signal = -0.99, p value = 1.0
Cell 11: depth 82.54um, signal = 0.0, p value = 1.0
Cell 51: depth 167.83um, signal = -0.6, p value = 1.0
Cell 223: depth 109.83um, signal = 0.0, p value = 1.0


In [54]:
sig = int_car4_bin - avg_bg_car4_bin
pv = pv_car43
max_sig = 0
all_cells = np.array(list(Cell.cell_data.keys())) - 1

cells = all_cells[np.logical_and(sig < max_sig, pv > 0.05)]
cells = cells[np.random.randint(0, high = len(cells) - 1, size = 10)]
for cell in cells:
    print('Cell {0}: depth {1}um, signal = {2}, p value = {3}'.format(cell, 
                                                                      np.round(depths[cell], decimals = 2), 
                                                                      np.round(sig[cell], decimals = 2), 
                                                                      np.round(pv[cell], decimals = 2)))


Cell 132: depth 148.87um, signal = -0.99, p value = 1.0
Cell 116: depth 248.77um, signal = -2.46, p value = 1.0
Cell 240: depth 292.06um, signal = -1.42, p value = 1.0
Cell 280: depth 188.9um, signal = -2.61, p value = 1.0
Cell 50: depth 225.07um, signal = -0.6, p value = 1.0
Cell 280: depth 188.9um, signal = -2.61, p value = 1.0
Cell 172: depth 208.21um, signal = -9.84, p value = 1.0
Cell 51: depth 167.83um, signal = -0.6, p value = 1.0
Cell 193: depth 254.59um, signal = -0.21, p value = 1.0
Cell 271: depth 338.76um, signal = -0.38, p value = 1.0


In [76]:
sig = int_car4_bin - avg_bg_car4_bin
pv = pv_car43
min_sig = 0
all_cells = np.array(list(Cell.cell_data.keys())) - 1

cells = all_cells[np.logical_and(sig > min_sig, pv > 0.05)]
cells = cells[np.random.randint(0, high = len(cells) - 1, size = 10)]
for cell in cells:
    print('Cell {0}: depth {1}um, signal = {2}, p value = {3}'.format(cell, 
                                                                      np.round(depths[cell], decimals = 2), 
                                                                      np.round(sig[cell], decimals = 2), 
                                                                      np.round(pv[cell], decimals = 2)))


Cell 62: depth 147.15um, signal = 0.5, p value = 0.5
Cell 62: depth 147.15um, signal = 0.5, p value = 0.5
Cell 62: depth 147.15um, signal = 0.5, p value = 0.5
Cell 62: depth 147.15um, signal = 0.5, p value = 0.5
Cell 62: depth 147.15um, signal = 0.5, p value = 0.5
Cell 62: depth 147.15um, signal = 0.5, p value = 0.5
Cell 62: depth 147.15um, signal = 0.5, p value = 0.5
Cell 62: depth 147.15um, signal = 0.5, p value = 0.5
Cell 62: depth 147.15um, signal = 0.5, p value = 0.5
Cell 62: depth 147.15um, signal = 0.5, p value = 0.5


## tac2+ cells

In [56]:
sig = int_tac2_bin - avg_bg_tac2_bin
pv = pv_tac23
max_sig = 10
all_cells = np.array(list(Cell.cell_data.keys())) - 1

cells = all_cells[np.logical_and(sig < max_sig, pv < 0.05)]
# cells = cells[np.random.randint(0, high = len(cells) - 1, size = 10)]
for cell in cells:
    print('Cell {0}: depth {1}um, signal = {2}, p value = {3}'.format(cell, 
                                                                      np.round(depths[cell], decimals = 2), 
                                                                      np.round(sig[cell], decimals = 2), 
                                                                      np.round(pv[cell], decimals = 2)))


Cell 249: depth 248.88um, signal = 9.08, p value = 0.0


## tac2- cells

In [57]:
sig = int_tac2_bin - avg_bg_tac2_bin
pv = pv_tac23
max_sig = 0
all_cells = np.array(list(Cell.cell_data.keys())) - 1

cells = all_cells[np.logical_and(sig < max_sig, pv > 0.05)]
cells = cells[np.random.randint(0, high = len(cells) - 1, size = 10)]
for cell in cells:
    print('Cell {0}: depth {1}um, signal = {2}, p value = {3}'.format(cell, 
                                                                      np.round(depths[cell], decimals = 2), 
                                                                      np.round(sig[cell], decimals = 2), 
                                                                      np.round(pv[cell], decimals = 2)))


Cell 163: depth 73.1um, signal = -6.96, p value = 1.0
Cell 163: depth 73.1um, signal = -6.96, p value = 1.0
Cell 163: depth 73.1um, signal = -6.96, p value = 1.0
Cell 163: depth 73.1um, signal = -6.96, p value = 1.0
Cell 163: depth 73.1um, signal = -6.96, p value = 1.0
Cell 163: depth 73.1um, signal = -6.96, p value = 1.0
Cell 163: depth 73.1um, signal = -6.96, p value = 1.0
Cell 163: depth 73.1um, signal = -6.96, p value = 1.0
Cell 163: depth 73.1um, signal = -6.96, p value = 1.0
Cell 163: depth 73.1um, signal = -6.96, p value = 1.0


In [58]:
sig = int_tac2_bin - avg_bg_tac2_bin
pv = pv_tac23
max_sig = 10
all_cells = np.array(list(Cell.cell_data.keys())) - 1

cells = all_cells[np.logical_and(sig < max_sig, pv > 0.05)]
cells = cells[np.random.randint(0, high = len(cells) - 1, size = 10)]
for cell in cells:
    print('Cell {0}: depth {1}um, signal = {2}, p value = {3}'.format(cell, 
                                                                      np.round(depths[cell], decimals = 2), 
                                                                      np.round(sig[cell], decimals = 2), 
                                                                      np.round(pv[cell], decimals = 2)))


Cell 241: depth 295.46um, signal = 0.0, p value = 1.0
Cell 110: depth 102.05um, signal = 0.0, p value = 1.0
Cell 264: depth 119.32um, signal = 0.0, p value = 1.0
Cell 135: depth 170.99um, signal = 0.0, p value = 1.0
Cell 167: depth 60.71um, signal = 0.0, p value = 1.0
Cell 38: depth 80.55um, signal = 0.0, p value = 1.0
Cell 198: depth 205.59um, signal = 0.0, p value = 1.0
Cell 54: depth 59.12um, signal = 0.0, p value = 1.0
Cell 156: depth 163.56um, signal = 0.0, p value = 1.0
Cell 90: depth 45.37um, signal = 0.0, p value = 1.0
